In [1]:
import pandas as pd
import numpy as np
import math as math
import scipy.stats as sts
import scipy.optimize as opt
import numpy.linalg as lin
import statsmodels.api as sm

In [2]:
#Opening files, weeks to use are from 173 to 224 both included

lnd_mov = pd.read_stata("Dataset/WLND.DTA")
lnd_upc = pd.read_stata("Dataset/UPCLND.DTA")
upc_f = pd.ExcelFile("Dataset/UPC_LIST_LND.xlsx")
upc_filtered = upc_f.parse("Sheet1")
demo = pd.read_stata("Dataset/demo.dta")
count = pd.read_stata("Dataset/ccount.dta")

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('float_format', '{:f}'.format)

In [5]:
# slicing data

lnd = lnd_mov.loc[lambda lnd: lnd_mov.week<225, :]
lndf = lnd.loc[lambda lnd2: lnd.week>172, :]
lndf_ok = lndf.loc[lambda lnd3: lndf.ok==1, :]

count2 = count.loc[:, lambda count: ['store', 'week', 'custcoun', 'date']]
count3 = count2.loc[lambda count2: count2.week>172, :]
countf = count3.loc[lambda count3: count3.week<225, :]

demof = demo.loc[:, lambda demo: ['store', 'zone', 'income', 'AGE60', 'ethnic', 'hvalmean', 'shopindx', 'cubdist', 'omnidist', 'sstrdist', 'dtdist']]

In [6]:
zones = demof.loc[lambda zones: demof.zone>=1,:]
zones.dropna()
zones.sort_values(by=['zone', 'store'])

lndf_ok.loc[:,'zone']=lndf_ok['store'].map(zones.set_index('store')['zone'])


C:\Users\Daniela\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Daniela\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [7]:
lndf_ok.loc[:,'income']=lndf_ok['store'].map(zones.set_index('store')['income'])
lndf_ok.loc[:,'AGE60']=lndf_ok['store'].map(zones.set_index('store')['AGE60'])
lndf_ok.loc[:,'ethnic']=lndf_ok['store'].map(zones.set_index('store')['ethnic'])
lndf_ok.loc[:,'hvalmean']=lndf_ok['store'].map(zones.set_index('store')['hvalmean'])
lndf_ok.loc[:,'shopindx']=lndf_ok['store'].map(zones.set_index('store')['shopindx'])
lndf_ok.loc[:,'cubdist']=lndf_ok['store'].map(zones.set_index('store')['cubdist'])
lndf_ok.loc[:,'omnidist']=lndf_ok['store'].map(zones.set_index('store')['omnidist'])
lndf_ok.loc[:,'sstrdist']=lndf_ok['store'].map(zones.set_index('store')['sstrdist'])
lndf_ok.loc[:,'dtdist']=lndf_ok['store'].map(zones.set_index('store')['dtdist'])
lndf_ok.describe()

,store,upc,week,move,qty,price,profit,ok,zone,income,AGE60,ethnic,hvalmean,shopindx,cubdist,omnidist,sstrdist,dtdist
count,825376.000000,825376.000000,825376.000000,825376.000000,825376.000000,825376.000000,825376.000000,825376.000000,741168.000000,741168.000000,741168.000000,741168.000000,741168.000000,741168.000000,741168.000000,741168.000000,741168.000000,741168.000000
mean,83.024143,3090875451.013073,198.941486,4.441770,1.000853,2.848114,9.395299,1.000000,5.317087,10.609513,0.174076,0.161130,144.189031,0.744808,6.307204,7.661776,5.264167,1.688274
std,36.451713,1467644637.095361,14.825661,17.814231,0.029193,3.501677,12.451626,0.000000,4.554855,0.285620,0.062847,0.192464,44.421616,0.230238,3.751742,5.357818,3.497469,0.500867
min,2.000000,1111135101.000000,173.000000,0.000000,1.000000,0.000000,-98.940000,1.000000,1.000000,9.867083,0.058054,0.024681,64.348363,0.000000,0.132097,0.966162,0.132097,0.213634
25%,54.000000,1111189226.000000,186.000000,0.000000,1.000000,0.000000,0.000000,1.000000,2.000000,10.456079,0.125798,0.043972,106.993573,0.730090,3.503788,4.701267,2.912922,1.473635
50%,89.000000,3700000991.000000,199.000000,1.000000,1.000000,2.090000,0.000000,1.000000,2.000000,10.620839,0.175542,0.074656,150.163751,0.818802,5.334974,6.521096,4.793439,1.702432
75%,113.000000,3700092714.000000,212.000000,5.000000,1.000000,4.990000,19.440000,1.000000,8.000000,10.793537,0.213949,0.190586,174.438833,0.889501,8.551712,8.657880,6.673879,1.894497
max,139.000000,20000100000.000000,224.000000,1992.000000,2.000000,22.670000,99.990000,1.000000,16.000000,11.236197,0.307398,0.995691,267.389720,0.986333,17.855951,31.404003,17.855951,3.000000


In [8]:
#adding description and volume
lndf_ok.loc[:,'desc']=lndf_ok['upc'].map(lnd_upc.set_index('upc')['descrip']).fillna(0)
lndf_ok.loc[:,'OZ']=lndf_ok['upc'].map(lnd_upc.set_index('upc')['size'])

C:\Users\Daniela\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Daniela\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
countf.loc[:,'store-week'] = countf['store'].apply(lambda x: x*1000)+countf['week']
countf_perweek = countf.groupby(['store','week','store-week'], as_index=False)['custcoun'].sum()

In [10]:
#lndf_ok=lndf_ok.drop(columns=['store-week','totcount'])
lndf_ok.loc[:,'store-week'] = lndf_ok['store'].apply(lambda x: x*1000)+lndf_ok['week']
lndf_ok.loc[:,'totcount']=lndf_ok['store-week'].map(countf_perweek.set_index('store-week')['custcoun'])

In [11]:
lndf_ok=lndf_ok.dropna()

In [153]:
lndf_ok.iloc[:,10:].describe()

,income,AGE60,ethnic,hvalmean,shopindx,cubdist,omnidist,sstrdist,dtdist,store-week,...,family,Tide,cheer,wisk,all_det,surf,share,64 OZ,128 OZ,nobrand
count,112835.000000,112835.000000,112835.000000,112835.000000,112835.000000,112835.000000,112835.000000,112835.000000,112835.000000,112835.000000,...,112835.000000,112835.000000,112835.000000,112835.000000,112835.000000,112835.000000,112835.000000,112835.000000,112835.000000,112835.000000
mean,10.610251,0.174567,0.159627,144.454679,0.745286,6.299275,7.649924,5.257331,1.691894,80776.538299,...,2471428.927638,0.355377,0.170036,0.199530,0.197430,0.077627,0.000780,0.667435,0.332565,0.000000
std,0.285920,0.062962,0.190532,44.422574,0.230907,3.726709,5.360975,3.462496,0.500807,35997.238761,...,1292787.601387,0.478630,0.375666,0.399649,0.398061,0.267584,0.001889,0.471134,0.471134,0.000000
min,9.867083,0.058054,0.024681,64.348363,0.000000,0.132097,0.966162,0.132097,0.213634,2173.000000,...,1111148.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000022,0.000000,0.000000,0.000000
25%,10.456079,0.125798,0.041542,106.993573,0.730090,3.503788,4.701267,2.912922,1.473635,53190.000000,...,1111187.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000178,0.000000,0.000000,0.000000
50%,10.620839,0.175542,0.074656,150.163751,0.818802,5.334974,6.521096,4.793439,1.702432,86214.000000,...,3700091.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000360,1.000000,0.000000,0.000000
75%,10.793537,0.213949,0.190586,174.438833,0.900589,8.551712,8.657880,6.673879,1.894497,111182.000000,...,3700092.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000726,1.000000,1.000000,0.000000
max,11.236197,0.307398,0.995691,267.389720,0.986333,17.855951,31.404003,17.855951,3.000000,134224.000000,...,3700097.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.061459,1.000000,1.000000,0.000000


In [12]:
#adding promotion dummy
promo_dummies = pd.get_dummies(lndf_ok['sale'])

In [13]:
promo_d= promo_dummies.B+promo_dummies.S+promo_dummies.G
promo_d=pd.DataFrame(promo_d)

In [14]:
promo_d.columns=['promo']
promo_d
lndf_ok.loc[:,'promo']=promo_d

In [15]:
lndf_ok.loc[:,'family']=round(lndf_ok.upc/1e3-0.499,0)
lndfbrand_d = pd.get_dummies(lndf_ok['family'])

In [16]:
#surf 1111142 1111148, wisk 1111187, all 1111189, cheer 3700093(28), tide 3700092(27)
a=lndfbrand_d.iloc[:,27:29]
b=lndfbrand_d.iloc[:,5:7]
c=lndfbrand_d.iloc[:,2]+lndfbrand_d.iloc[:,4]
lndfbrand_d2 = pd.concat([a,b,c], axis=1)
lndfbrand_d2.columns=['Tide','cheer','wisk','all_det','surf']

In [17]:
lndf_ok=pd.concat([lndf_ok,lndfbrand_d2], axis=1)
lndf_ok['share'] = lndf_ok.move/lndf_ok.totcount

In [18]:
size_dummies_lnd = pd.get_dummies(lndf_ok['OZ'])
list(size_dummies_lnd)
#size_dummies_lnd['64 OZ']
size_64_d_lnd = size_dummies_lnd['64 OZ']
lndf_ok=pd.concat([lndf_ok,size_64_d_lnd], axis=1)

In [19]:
lndf_ok.rename(columns={'64 OZ':'64_OZ'})
""

''

In [20]:
lndf_ok = lndf_ok.loc[lambda lnd3: lndf_ok.move>0, :]

In [21]:
Tide64 = upc_filtered.iloc[:11,:]
Tide128 = upc_filtered.iloc[11:35,:]
Cheer64 = upc_filtered.iloc[35:45,:]
Cheer128 = upc_filtered.iloc[45:56,:]
All_d64 = upc_filtered.iloc[56:76,:]
All_d128 = upc_filtered.iloc[76:101,:]
Surf64 = upc_filtered.iloc[101:109,:]
Surf128 = upc_filtered.iloc[109:122,:]
Wisk64 = upc_filtered.iloc[122:141,:]
Wisk128 = upc_filtered.iloc[141:,:]

In [22]:
Surf64_d1=lndf_ok['upc'].isin(Surf64['upc'])
Surf64_d2=pd.get_dummies(Surf64_d1)
Surf128_d1=lndf_ok['upc'].isin(Surf128['upc'])
Surf128_d2=pd.get_dummies(Surf128_d1)
lndf_ok['surf'] = Surf64_d2.iloc[:,1]+Surf128_d2.iloc[:,1]
print("surf64: ",(Surf64_d2.iloc[:,1]*lndf_ok['move']).sum())
print("surf128: ",(Surf128_d2.iloc[:,1]*lndf_ok['move']).sum())

surf64:  86347
surf128:  14891


In [23]:
All64_d1=lndf_ok['upc'].isin(All_d64['upc'])
All64_d2=pd.get_dummies(All64_d1)
All128_d1=lndf_ok['upc'].isin(All_d128['upc'])
All128_d2=pd.get_dummies(All128_d1)
lndf_ok['all_det'] = All64_d2.iloc[:,1]+All128_d2.iloc[:,1]
print("All64: ",(All64_d2.iloc[:,1]*lndf_ok['move']).sum())
print("All128: ",(All128_d2.iloc[:,1]*lndf_ok['move']).sum())

All64:  117628
All128:  75671


In [24]:
Wisk64_d1=lndf_ok['upc'].isin(Wisk64['upc'])
Wisk64_d2=pd.get_dummies(Wisk64_d1)
Wisk128_d1=lndf_ok['upc'].isin(Wisk128['upc'])
Wisk128_d2=pd.get_dummies(Wisk128_d1)
lndf_ok['wisk'] = Wisk64_d2.iloc[:,1]+Wisk128_d2.iloc[:,1]
print("wisk64: ",(Wisk64_d2.iloc[:,1]*lndf_ok['move']).sum())
print("Wisk128: ",(Wisk128_d2.iloc[:,1]*lndf_ok['move']).sum())

wisk64:  216856
Wisk128:  68267


In [25]:
Cheer64_d1=lndf_ok['upc'].isin(Cheer64['upc'])
Cheer64_d2=pd.get_dummies(Cheer64_d1)
Cheer128_d1=lndf_ok['upc'].isin(Cheer128['upc'])
Cheer128_d2=pd.get_dummies(Cheer128_d1)
lndf_ok['cheer'] = Cheer64_d2.iloc[:,1]+Cheer128_d2.iloc[:,1]
print("Cheer64: ",(Cheer64_d2.iloc[:,1]*lndf_ok['move']).sum())
print("Cheer128: ",(Cheer128_d2.iloc[:,1]*lndf_ok['move']).sum())

Cheer64:  112327
Cheer128:  58206


In [26]:
Tide64_d1=lndf_ok['upc'].isin(Tide64['upc'])
Tide64_d2=pd.get_dummies(Tide64_d1)
Tide128_d1=lndf_ok['upc'].isin(Tide128['upc'])
Tide128_d2=pd.get_dummies(Tide128_d1)
lndf_ok['Tide'] = Tide64_d2.iloc[:,1]+Tide128_d2.iloc[:,1]
#lndf_ok.Tide.describe()
(lndf_ok['Tide']*lndf_ok['move']).sum()
#test=lndf_ok.loc[lambda lnd3: lndf_ok.upc==3700091240, :]
#test.upc.describe()
print("Tide64: ",(Tide64_d2.iloc[:,1]*lndf_ok['move']).sum())
print("Tide128: ",(Tide128_d2.iloc[:,1]*lndf_ok['move']).sum())

Tide64:  305936
Tide128:  409384


In [27]:
lndf_ok['64 OZ']=Surf64_d2.iloc[:,1]+Tide64_d2.iloc[:,1]+Cheer64_d2.iloc[:,1]+Wisk64_d2.iloc[:,1]+All64_d2.iloc[:,1]
lndf_ok['128 OZ']=Surf128_d2.iloc[:,1]+Tide128_d2.iloc[:,1]+Cheer128_d2.iloc[:,1]+Wisk128_d2.iloc[:,1]+All128_d2.iloc[:,1]
lndf_ok['nobrand']=1-lndf_ok['64 OZ']-lndf_ok['128 OZ'] #nobrand=1 other brands
#lndf_ok.drop(columns=['Cheer'], axis=1)

In [28]:
lndf_ok = lndf_ok.loc[lambda lndnb: lndf_ok.nobrand==0, :]

In [29]:
tide=lndf_ok.move*lndf_ok.Tide
surf=lndf_ok.move*lndf_ok.surf
all_d=lndf_ok.move*lndf_ok.all_det
cheer=lndf_ok.move*lndf_ok.cheer
wisk=lndf_ok.move*lndf_ok.wisk
total=tide.sum()+surf.sum()+all_d.sum()+cheer.sum()+wisk.sum()

In [30]:
print(tide.sum()/total)
print(surf.sum()/total)
print(all_d.sum()/total)
print(cheer.sum()/total)
print(wisk.sum()/total)

0.48810211850730767
0.06908024698518539
0.13189852290631335
0.11636403088884234
0.19455508071235122


In [31]:
lndf_ok.shape

(112835, 34)

In [157]:
#Price, and cost, Descriptive Statistics

AvgP_s64 = np.sum(lndf_ok['surf']*lndf_ok['64 OZ']*lndf_ok['price']*lndf_ok['move'])/np.sum(lndf_ok['surf']*lndf_ok['64 OZ']*lndf_ok['move'])
AvgC_s64 = np.sum(lndf_ok['surf']*lndf_ok['64 OZ']*(lndf_ok['price']*(1-lndf_ok['profit']/100))*lndf_ok['move'])/np.sum(lndf_ok['surf']*lndf_ok['64 OZ']*lndf_ok['move'])

AvgP_t64 = np.sum(lndf_ok['Tide']*lndf_ok['64 OZ']*lndf_ok['price']*lndf_ok['move'])/np.sum(lndf_ok['Tide']*lndf_ok['64 OZ']*lndf_ok['move'])
AvgC_t64 = np.sum(lndf_ok['Tide']*lndf_ok['64 OZ']*(lndf_ok['price']*(1-lndf_ok['profit']/100))*lndf_ok['move'])/np.sum(lndf_ok['Tide']*lndf_ok['64 OZ']*lndf_ok['move'])

AvgP_t128 = np.sum(lndf_ok['Tide']*lndf_ok['128 OZ']*lndf_ok['price']*lndf_ok['move'])/np.sum(lndf_ok['Tide']*lndf_ok['128 OZ']*lndf_ok['move'])
AvgC_t128 = np.sum(lndf_ok['Tide']*lndf_ok['128 OZ']*(lndf_ok['price']*(1-lndf_ok['profit']/100))*lndf_ok['move'])/np.sum(lndf_ok['Tide']*lndf_ok['128 OZ']*lndf_ok['move'])

AvgP_c64 = np.sum(lndf_ok['cheer']*lndf_ok['64 OZ']*lndf_ok['price']*lndf_ok['move'])/np.sum(lndf_ok['cheer']*lndf_ok['64 OZ']*lndf_ok['move'])
AvgC_c64 = np.sum(lndf_ok['cheer']*lndf_ok['64 OZ']*(lndf_ok['price']*(1-lndf_ok['profit']/100))*lndf_ok['move'])/np.sum(lndf_ok['cheer']*lndf_ok['64 OZ']*lndf_ok['move'])

AvgP_c128 = np.sum(lndf_ok['cheer']*lndf_ok['128 OZ']*lndf_ok['price']*lndf_ok['move'])/np.sum(lndf_ok['cheer']*lndf_ok['128 OZ']*lndf_ok['move'])
AvgC_c128 = np.sum(lndf_ok['cheer']*lndf_ok['128 OZ']*(lndf_ok['price']*(1-lndf_ok['profit']/100))*lndf_ok['move'])/np.sum(lndf_ok['cheer']*lndf_ok['128 OZ']*lndf_ok['move'])

AvgP_w64 = np.sum(lndf_ok['wisk']*lndf_ok['64 OZ']*lndf_ok['price']*lndf_ok['move'])/np.sum(lndf_ok['wisk']*lndf_ok['64 OZ']*lndf_ok['move'])
AvgC_w64 = np.sum(lndf_ok['wisk']*lndf_ok['64 OZ']*(lndf_ok['price']*(1-lndf_ok['profit']/100))*lndf_ok['move'])/np.sum(lndf_ok['wisk']*lndf_ok['64 OZ']*lndf_ok['move'])

AvgP_w128 = np.sum(lndf_ok['wisk']*lndf_ok['128 OZ']*lndf_ok['price']*lndf_ok['move'])/np.sum(lndf_ok['wisk']*lndf_ok['128 OZ']*lndf_ok['move'])
AvgC_w128 = np.sum(lndf_ok['wisk']*lndf_ok['128 OZ']*(lndf_ok['price']*(1-lndf_ok['profit']/100))*lndf_ok['move'])/np.sum(lndf_ok['wisk']*lndf_ok['128 OZ']*lndf_ok['move'])

AvgP_a64 = np.sum(lndf_ok['all_det']*lndf_ok['64 OZ']*lndf_ok['price']*lndf_ok['move'])/np.sum(lndf_ok['all_det']*lndf_ok['64 OZ']*lndf_ok['move'])
AvgC_a64 = np.sum(lndf_ok['all_det']*lndf_ok['64 OZ']*(lndf_ok['price']*(1-lndf_ok['profit']/100))*lndf_ok['move'])/np.sum(lndf_ok['all_det']*lndf_ok['64 OZ']*lndf_ok['move'])

AvgP_a128 = np.sum(lndf_ok['all_det']*lndf_ok['128 OZ']*lndf_ok['price']*lndf_ok['move'])/np.sum(lndf_ok['all_det']*lndf_ok['128 OZ']*lndf_ok['move'])
AvgC_a128 = np.sum(lndf_ok['all_det']*lndf_ok['128 OZ']*(lndf_ok['price']*(1-lndf_ok['profit']/100))*lndf_ok['move'])/np.sum(lndf_ok['all_det']*lndf_ok['128 OZ']*lndf_ok['move'])


print("Avg. Prices: s64: ", AvgP_s64, "t64",AvgP_t64 , "t128", AvgP_t128, "c64", AvgP_c64, "c128", AvgP_c128, "w64", AvgP_w64, "w128", AvgP_w128, "a64", AvgP_a64, "a128", AvgP_a128)
print("Avg. Costs: s64: ", AvgC_s64, "t64",AvgC_t64 , "t128", AvgC_t128, "c64", AvgC_c64, "c128", AvgC_c128, "w64", AvgC_w64, "w128", AvgC_w128, "a64", AvgC_a64, "a128", AvgC_a128)

Avg. Prices: s64:  3.8014602707679472 t64 3.2379190418911143 t128 7.528759160103961 c64 3.4926801214311785 c128 7.215110297907431 w64 4.319994189692699 w128 7.432468689117729 a64 3.3245736559322605 a128 5.618716945725576
Avg. Costs: s64:  3.830799163225127 t64 2.981880172065399 t128 7.260698144031032 c64 3.048561545558949 c128 6.518187494536646 w64 4.163419629509906 w128 7.201873326160517 a64 3.1886047327081988 a128 5.1545732543158325


In [32]:
def InstVar(price, instruments):
    first_stage = sm.OLS(price,instruments).fit()
    coeffs = first_stage.params
    price_est = instruments@coeffs
    price_estf = np.reshape(price_est,(price_est.size,1))
    return price_estf

# Estimation of demand

In [33]:
"""
Variables to be estimated:
    
alpha => alpha_j (with j for each brand), mean intrinsic utility across consumers for brand and size x

sigma => sigma_income, sigma_age, sigma_ethnic, sigma_hval, sigma_shopindx, sigma_edlpd: deviations 
from the mean due to demographic variables (common values across stores)

beta => beta_promo, beta_size: mean coefficients for observed attributes
    
lambda_p&s => lambda_promo, lambda_size: deviation coefficients for for observed attributes
    
theta and gamma => theta_price, gamma_income, gamma_age, gamma_ethnic, gamma_hval, gamma_shopindx, gamma_edlpd: coefficients for price
    
lambda_price => lambda_price: coefficient for the deviation of price

l_matrix => l1_j, l2_j = coefficients of the latent attributes for each brand 

draws are relative to the standard deviations coefficients
"""
print( )

In [34]:
"""main equations:
    
delta(alpha, beta, sigma, gamma, promo and size, demog_vars, brand_dummies, price, xi): this function calculates the mean utility 
for each brand and store. 

delta_jts = brand_dummies_ts@alpha_j+ promo&size@beta + demog_vars_s@sigma \
            - (theta+demog_vars_s@gamma)*price_jts + xi // vec. of obsx1

mu(promo&size, price, lambda_p&s, lambda_price, l_matrix, simulation draws): this function calculates the deviations from 
the mean, using independent draws."""
print( )

In [36]:
def delta_fun (alpha, beta, sigma, gamma, theta, xi, data_promo_size, data_demog, brand_dummies, price, store_week):
    a = np.reshape(alpha,(alpha.size,1))
    b = np.reshape(beta,(beta.size,1))
    s = np.reshape(sigma,(sigma.size,1))
    g = np.reshape(gamma,(gamma.size,1))
    ps = data_promo_size
    dd = data_demog
    bd = brand_dummies
    p = price
    mkt = np.reshape(store_week,(store_week.size,1))
    
    delta_mat = np.zeros((mkt.shape[0], 2))
    delta_mat[:,0] = mkt[:,0]
    trd = dd@s + bd@a +ps@b + (theta+dd@g)*p    
    delta_mat[:,1] = trd[:,0]
    return delta_mat

def mu_fun(data_promo_size, price, brand_dummies, L_matrix, lambda_x,lambda_price, sim_beta, sim_theta, sim_w):
    
    ps = np.reshape(data_promo_size,(data_promo_size.shape[0],data_promo_size.shape[1]))
    p = np.reshape(price,(price.size,1))
    bd = brand_dummies

    lambda_x = np.reshape(lambda_x,(lambda_x.size,1))
    L_matrix = L_matrix
    
    sim_beta = sim_beta
    sim_theta = np.reshape(sim_theta,(sim_theta.size,1))
    sim_w = sim_w
    
    sim_mat = np.hstack((sim_beta*lambda_x.T, lambda_price*sim_theta,sim_w@L_matrix.T)) # S x (J+3) matrix
    param_mat = np.hstack((ps,price,brand_dummies))  # obs x (J+3) matrix
    
    mu_mat = param_mat@sim_mat.T #obsxS
    return mu_mat

In [37]:
#contraction mapping

def est_ms_cm(delta1, theta2, data, simP):
    sim_beta, sim_theta, sim_w = simP
    data_promo_size, data_demog, brand_dummies, price, store_week, share = data
    L_matrix, lambda_x,lambda_price = theta2

    d=delta1
    mu = mu_fun(data_promo_size, price, brand_dummies,L_matrix, lambda_x,lambda_price,sim_beta, sim_theta, sim_w) # obs x S

    delta = d[:,1]
    delta = np.reshape(np.asarray(delta),(price.size,1))
    share = np.reshape(np.asarray(share),(share.size,1))
    delta = np.exp(delta)
    mu = np.exp(mu)
    market = np.reshape(store_week,(store_week.size,1))

    share_mat = delta*mu
    share_mat = np.hstack((market,share_mat))
    share_mat2 = pd.DataFrame((share_mat))
    
    tot_share_mat = share_mat2.groupby(share_mat2.iloc[:,0]).sum()
    tot_share_mat = tot_share_mat + 1
    
    share_mat2=share_mat2.rename(columns={0:'market'})
    share_mat2.sort_values(by=['market'])
    
    r2=share_mat2.iloc[0,0]
    a=(tot_share_mat.loc[r2,1:]).values.reshape(1,30)
    i=0
    for i in range(share_mat2.shape[0]):
        r=share_mat2.iloc[i,0]
        if r==r2:
            share_mat2.iloc[i,1:]=(share_mat2.iloc[i,1:]).values.reshape(1,30)/a #tot_share_mat.loc[r,:]
        else:
            r2=share_mat2.iloc[i,0]
            a=(tot_share_mat.loc[r2,1:]).values.reshape(1,30)
            share_mat2.iloc[i,1:]=(share_mat2.iloc[i,1:]).values.reshape(1,30)/a
    
    ns = mu.shape[1]
    est_shares_vec = np.zeros((d.shape[0],1))
    est_shares_vec[:,0] = (share_mat2.iloc[:,1:].sum(axis=1)).apply(lambda x: x*(1/ns))
    new_delta_3 = d[:,1] + np.log(share[:,0]) - np.log(est_shares_vec[:,0])
    d[:,1] = new_delta_3
    new_delta = d
    return new_delta, est_shares_vec

In [38]:
#This function generates the new deltas for the iteration.

def delta_eq(d_in,theta2, data, simP):
    data_promo_size, data_demog, brand_dummies, price, store_week, share = data
 
    delta_new, estim_shares_new = est_ms_cm(d_in, theta2, data, simP) # estimated shares are not used in this cell.

    diff_sh = abs(np.log(share) - np.log(estim_shares_new))
    epsilon = 1e-8
    i=0
    while diff_sh.all() >= epsilon:
        delta_ant=delta_new
        delta_new, est_shares_new = est_ms_cm(delta_ant, theta2, data, simP)
        diff_sh = abs(np.log(share) - np.log(est_shares_new))
        print(diff_sh.max())
        i+=1

    delta_eq = delta_new
    
    return delta_eq

In [39]:
#this function calculates the error vector (xi_jt)

def errorvec(delta_inic, data, theta1, theta2, xi, simP):
    
    alpha, beta, sigma, gamma, theta = theta1
    data_promo_size, data_demog, brand_dummies, price, store_week, share = data
    delta_equiv = delta_eq(delta_inic, theta2,data, simP)

    delta_mod = delta_fun(alpha, beta, sigma, gamma, theta, xi, data_promo_size, data_demog, brand_dummies, price, store_week)

    error_vec = delta_equiv[:,1] - delta_mod[:,1]
    
    return error_vec

In [40]:
def logs(market, share_obs):
    storeweek = np.array(market)
    share = np.array(share_obs)
    ts=np.zeros((share.shape[0],1))
    share2=np.hstack((storeweek,share,ts))
    share2=pd.DataFrame((share2))
    tot_shares_real=share2.groupby(share2.iloc[:,0]).sum()
    i=0
    for i in range(share.shape[0]):
        r=share2.iloc[i,0]
        tots=tot_shares_real.loc[r,:]
        share2.iloc[i,2]=1-tots[1]
        i+=1
    logs=np.asarray(np.log(share2.iloc[:,1])-np.log(share2.iloc[:,2]))
    logs=np.reshape(logs, (logs.size,1))
    return logs

In [41]:
def paramsnohet(datax, theta1):
    alpha, beta, sigma, gamma, theta = theta1
    data_promo_size, data_demog, brand_dummies, price, store_week, share = datax
    
    a = np.reshape(alpha,(alpha.size,1))
    b = np.reshape(beta,(beta.size,1))
    s = np.reshape(sigma,(sigma.size,1))
    g = np.reshape(gamma,(gamma.size,1))
    ps = data_promo_size
    dd = data_demog
    bd = brand_dummies
    p = price
    mkt = np.reshape(store_week,(store_week.size,1))
    
    Y = logs(mkt, share)
    X=np.hstack((ps, dd, p, bd, dd*p))
    X=pd.DataFrame(X)
    X.columns=['promo', '64 OZ', 'income', 'AGE60', 'ethnic', 'hvalmean','shopindx','sstrdist', 'price_e',\
              'Tide','cheer','wisk','all','surf', 'inc*p', 'AGE*p', 'eth*p', 'hval*p', 'shop*p', 'EDLP*p']
    
    trd = dd@s + bd@a +ps@b + (theta+dd@g)*p
    
    par_nohet = sm.OLS(Y,X).fit()
    return trd.shape, par_nohet.summary()

In [42]:
#GMM code
def GMM_criterion(params, *args):
    del_in, data, xi, simP, instruments = args
    a1,a2,a3,a4,a5,b1,b2,s1,s2,s3,s4,s5,s6,g1,g2,g3,g4,g5,g6, theta,\
    L11,L21,L22,L31,L32,L41,L42,L51,L52,lx1,lx2,lambda_price = params
    data_promo_size, data_demog, brand_dummies, price, store_week, share = data
    
    alpha=np.array([a1,a2,a3,a4,a5])
    beta=np.array([b1,b2])
    gamma = np.array([g1,g2,g3,g4,g5,g6])
    sigma = np.array([s1,s2,s3,s4,s5,s6])
    L_matrix=np.array([[L11,0],[L21,L22],[L31,L32],[L41,L42],[L51,L52]])
    lambda_x = np.array([lx1,lx2])
    
    theta1=(alpha, beta, sigma, gamma, theta)
    theta2=(L_matrix, lambda_x, lambda_price)
    Z=instruments
    mat_inv = lin.inv(Z.T@Z)
    error_vec = errorvec(del_in, data, theta1, theta2, xi, simP)
    GMM = error_vec.T@Z@mat_inv@Z.T@error_vec
    print("GMM: ",GMM)
    return GMM

### List of parameters, variables and draws:

#### Case 1: Laundry Detergent

- Brands: Tide, Surf, All, Cheer, Wisk
- Sizes: 64 OZ and 128 OZ

##### Variables:

- price: rows: # obs, columns: weekly price per product per store
- data_demog: rows: # obs, columns: income, age, ethnic, hval, shopindx, edlpd
- brand_dummies: rows: # obs, columns: 5 columns of dummies for Tide, Surf, All, Cheer, Wisk respectively
- data_promo_size: rows: # obs, columns: 2 columns of dummies for promo (1 if the price includes promo) and size (64 OZ = 1)
- storeweek: rows: # obs, columns: ID for each market (built from original columns (store x 1,000) + week)
- share: rows: # obs, columns: absolute share (also considering outside opt) for each product in each market.

##### Parameters:

- alpha: rows: 5, columns: 1 => alpha_tide, alpha_surf, alpha_all, alpha_cheer, alpha_wisk
- beta: rows: 2, columns: 1 => beta_promo, beta_size
- sigma: rows: 6, columns: 1 => sigma_income, sigma_age, sigma_ethnic, sigma_hval, sigma_shopindx, sigma_edlpd
- gamma: rows: 6, columns: 1 => gamma_income, gamma_age, gamma_ethnic, gamma_hval, gamma_shopindx, gamma_edlpd
- theta: 1 parameter, scalar
- L_matrix: rows: 5, columns: 2 => [[L1_tide, L2_tide],[L1_surf, L2_surf], [L1_all, L2_all], [L1_cheer, L2_cheer], [L1_wisk, L2_wisk]]. L1_tide arbitrarily set to 1, fixed to 0.
- lambda_x: rows: 2, columns: 1 => lambda_promo, lambda_size
- lambda_price: 1 parameter, scalar

##### Draws:

Normal Standard draws.

- sim_beta: rows: #Draws, columns: 2 => sim_beta_promo, sim_beta_size. Draws for the standard deviation of promo and size 
- sim_theta: rows: #Draws, columns: 1 => sim_theta. Draws for the theta parameter relative to price
- sim_w: rows: #Draws, columns: 2 => sim_w1, sim_w2. Draws for the w vector used to calculate the Sigma matrix

In [43]:
# Creating uniform and standard normal draws

sim_draws_beta = 30
sim_draws_theta = 30
sim_draws_w = 30
np.random.seed(1979)

uniform_beta = sts.uniform.rvs(0, 1, size=(sim_draws_beta,2))
normal_beta_draws = sts.norm.ppf(uniform_beta, loc=0, scale=1)

uniform_theta = sts.uniform.rvs(0, 1, size=(sim_draws_theta, 1))
normal_theta_draws = sts.norm.ppf(uniform_theta, loc=0, scale=1)

uniform_w = sts.uniform.rvs(0, 1, size=(sim_draws_w, 2))
normal_w_draws = sts.norm.ppf(uniform_w, loc=0, scale=1)

print(normal_beta_draws.shape, normal_theta_draws.shape, normal_w_draws.shape)

(30, 2) (30, 1) (30, 2)


In [44]:
def delta_ini(market, share_obs):
    storeweek = np.array(market)
    share = np.array(share_obs)
    ts=np.zeros((share.shape[0],1))
    share2=np.hstack((storeweek,share,ts))
    share2=pd.DataFrame((share2))
    tot_shares_real=share2.groupby(share2.iloc[:,0]).sum()
    i=0
    for i in range(share.shape[0]):
        r=share2.iloc[i,0]
        tots=tot_shares_real.loc[r,:]
        share2.iloc[i,2]=1-tots[1]
        i+=1
    logs=np.asarray(np.log(share2.iloc[:,1])-np.log(share2.iloc[:,2]))
    logs=np.reshape(logs, (logs.size,1))
    delta_in=np.hstack((storeweek,logs))
    return delta_in

In [45]:
# preparing the data, grouping it by store-week

demog = np.array(lndf_ok[['income', 'AGE60', 'ethnic', 'hvalmean','shopindx','sstrdist']])
brands = np.array(lndf_ok[['Tide','cheer','wisk','all_det','surf']])
price = np.array(np.log(lndf_ok[['price']]))
wholeprice = np.log(np.array(lndf_ok[['price']])*(1-np.array(lndf_ok[['profit']])/100))
promo=np.reshape(lndf_ok[['promo']],(lndf_ok[['promo']].shape[0],1))
instr=np.hstack((promo,demog, wholeprice, brands))
price_est=InstVar(price,instr)

datapack_GMM = np.hstack((price_est, wholeprice, lndf_ok[['promo', '64 OZ', '128 OZ', 'move','totcount', 'income', 'AGE60',\
                                                          'ethnic', 'hvalmean','shopindx','sstrdist', 'Tide','cheer','wisk'\
                                                          ,'all_det','surf', 'price', 'store-week', 'share', 'store', 'zone', 'week']]))

datapack_GMM=pd.DataFrame(datapack_GMM)
datapack_GMM.columns = ['Price_est', 'wholeprice', 'promo', '64 OZ', '128 OZ', 'move','totcount', 'income', 'AGE60',\
                        'ethnic', 'hvalmean','shopindx','sstrdist', 'Tide','cheer','wisk','all_det',\
                        'surf', 'price', 'store-week', 'share', 'store', 'zone', 'week']
datapack_GMM['64_OZ(2)']=datapack_GMM[['64 OZ']]
datapack_GMM['Price_est']=datapack_GMM['Price_est']*datapack_GMM['move']
datapack_GMM['wholeprice']=datapack_GMM['wholeprice']*datapack_GMM['move']
datapack_GMM = datapack_GMM.groupby(['store-week', 'Tide', 'cheer', 'wisk', 'all_det', 'surf', '64 OZ'], as_index=False).sum()

datapack_GMM['Price_est'] =datapack_GMM['Price_est']/(datapack_GMM['move']) 
datapack_GMM['wholeprice'] =datapack_GMM['wholeprice']/(datapack_GMM['move']) 
datapack_GMM['totcount'] =datapack_GMM['totcount']/(datapack_GMM['128 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['income'] =datapack_GMM['income']/(datapack_GMM['128 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['AGE60'] =datapack_GMM['AGE60']/(datapack_GMM['128 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['ethnic'] =datapack_GMM['ethnic']/(datapack_GMM['128 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['hvalmean'] =datapack_GMM['hvalmean']/(datapack_GMM['128 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['shopindx'] =datapack_GMM['shopindx']/(datapack_GMM['128 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['sstrdist'] =datapack_GMM['sstrdist']/(datapack_GMM['128 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['price'] =datapack_GMM['price']/(datapack_GMM['128 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['store'] =datapack_GMM['store']/(datapack_GMM['128 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['zone'] =datapack_GMM['zone']/(datapack_GMM['128 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['week'] =datapack_GMM['week']/(datapack_GMM['128 OZ']+datapack_GMM['64_OZ(2)']) 

i=0
for i in range(datapack_GMM.shape[0]):
    if datapack_GMM.loc[i,'promo']>0:
        datapack_GMM.loc[i,'promo'] = 1
    else:
        pass
    i+=1


In [46]:
promosize = np.array(datapack_GMM[['promo', '64 OZ']])
demog = np.array(datapack_GMM[['income', 'AGE60', 'ethnic', 'hvalmean','shopindx','sstrdist']])
brands = np.array(datapack_GMM[['Tide','cheer','wisk','all_det','surf']])
price = np.array(datapack_GMM[['price']])#.astype(np.float16)
storeweek = np.array(datapack_GMM[['store-week']])
share = np.array(datapack_GMM[['share']])
price_est=np.array(datapack_GMM[['Price_est']])
promo = np.array(datapack_GMM[['promo']])
wholeprice = np.array(datapack_GMM[['wholeprice']])

delta_in=delta_ini(storeweek,share)

In [47]:
data = (promosize, demog, brands, price_est, storeweek, share)

alpha=np.array([0,0,0,0,0])
beta=np.array([0,0])
gamma = np.array([0,0,0,0,0,0])
sigma = np.array([0,0,0,0,0,0])
theta = 0

theta1=(alpha, beta, sigma, gamma, theta)

results_nohet=paramsnohet(data, theta1)

In [48]:
results_nohet

((35360, 1), <class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                      y   R-squared:                       0.515
 Model:                            OLS   Adj. R-squared:                  0.515
 Method:                 Least Squares   F-statistic:                     1977.
 Date:                Fri, 22 Mar 2019   Prob (F-statistic):               0.00
 Time:                        10:06:40   Log-Likelihood:                -45655.
 No. Observations:               35360   AIC:                         9.135e+04
 Df Residuals:                   35340   BIC:                         9.152e+04
 Df Model:                          19                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------

In [49]:
a1=0.6922
a2=-0.5926
a3=-0.6875
a4=-0.9222
a5=-1.2102
b1=0.7338
b2=0.4730
s1=-0.5265
s2=-1.2057
s3=-0.4378
s4=0.0004
s5=0.1349
s6=-0.0126
g1=0.4136
g2=0.5516
g3=0.0596
g4=0.0005
g5= 0.1153
g6=-0.0088
L11= 0.15006862 #L12 arbitrarily set to zero
L21= 0.26061014 
L22= 0.05842542 
L31= 0.25786418 
L32= 0.11017376
L41= 0.29917304 
L42= 0.08253374 
L51= 0.33784066 
L52= 0.15487346 
lx1= 0.01303555
lx2= 0.12150025 
theta = -5.3350
lambda_price = 0.44632901

alpha=np.array([a1,a2,a3,a4,a5])
beta=np.array([b1,b2])
gamma = np.array([g1,g2,g3,g4,g5,g6])
sigma = np.array([s1,s2,s3,s4,s5,s6])

#theta1=(alpha, beta, sigma, gamma, theta)


simP = (normal_beta_draws, normal_theta_draws, normal_w_draws)

params = np.array([a1,a2,a3,a4,a5,b1,b2,s1,s2,s3,s4,s5,s6,g1,g2,g3,g4,g5,g6,theta,\
                   L11,L21,L22,L31,L32,L41,L42,L51,L52,lx1,lx2,lambda_price])
#
data = (promosize, demog, brands, price_est, storeweek, share)
xi = np.zeros((datapack_GMM.shape[0],1))

instruments=np.hstack((promo,demog, wholeprice, brands))

args = (delta_in, data, xi, simP, instruments)
Bounds = ((None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None),\
          (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None),\
          (None, None), (None, None), (None, None), (None, None), (None, None), (None, None),\
          (1e-10, None), (1e-10, None), (1e-10, None), (1e-10, None), (1e-10, None), (1e-10, None),\
          (1e-10, None), (1e-10, None), (1e-10, None), (1e-10, None), (1e-10, None), (1e-10, None))

In [60]:
results = opt.minimize(GMM_criterion, params, args=(args), method='SLSQP', bounds=Bounds, tol=1e-5) #options={'maxiter': 1}

0.010899671999583305
0.0011989725508918525
0.00013247928681003884
1.4646358623693345e-05
GMM:  366.4404735616838
GMM:  366.44046650549205
GMM:  366.4404662721572
GMM:  366.4404661792913
GMM:  366.4404661053818
GMM:  366.4404662565964
GMM:  366.44046614756587
GMM:  366.44046613666586
GMM:  366.4404661590494
GMM:  366.44048121784874
GMM:  366.44046599794297
GMM:  366.44045522817316
GMM:  366.44137558165806
GMM:  366.44047715918026
GMM:  366.44009326198056
GMM:  366.44049148697707
GMM:  366.4404659114335
GMM:  366.44044802069135
GMM:  366.44197488823477
GMM:  366.4404845472394
GMM:  366.4398473851665
GMM:  366.4404665594617
GMM:  366.4404661223921
1.4233059175694507e-10
1.5194956404229742e-11
GMM:  366.4404661390997
GMM:  366.44046614398246
GMM:  366.44046612339133
GMM:  366.440466146938
GMM:  366.4404661533243
GMM:  366.44046611826
GMM:  366.4404661319384
GMM:  366.44046614438366
GMM:  366.44046609462004
3.984332863637974e-10
4.0884629015636165e-11
GMM:  366.4404661337274
8.5945472960702

C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide


8.981597865047297
8.980742446991414
8.928493842118423
8.815673786712784
8.809425716063204
8.80942272932846
8.805897334113421
8.695189319087639
8.693190083126838
8.693179945194322
8.693171957890375
8.693156035214807
8.663047328218592
8.575429513369306
8.575366902401852
8.575364792133653
8.575363441523805
8.575362535505088
8.575361870224738
8.575361295996874
8.575360683619547
8.575359879383686
8.575357995520623
8.575354373266315
8.575347205774104
8.575332911990092
8.57530420737394
8.575210616212585
8.574797821873
8.458587873817429
8.438986983799724
8.427990471899973
8.271940140987654
8.24780592288596
8.209838123260042
8.169090140332694
8.135354379392343
8.108130729056928
8.084048635230387
8.061132646659985
7.972264739679966
7.7785844507518895
GMM:  9765619899469710.0
1.2447382202396362
1.0214201153750584
0.9425778028410652
0.9074739032561339
0.8954529101582254
0.8884044462078484
0.8805433277781334
0.8710241034376227
0.860558304630314
GMM:  97682535831586.6
0.6367252543425206
0.2372756396

1.5624834759364603e-10
GMM:  0.14193775353532245
GMM:  0.14193767745338481
GMM:  0.1419377366372373
GMM:  0.14193769531424194
GMM:  0.14193773174382407
GMM:  0.1419377074149224
GMM:  0.14193777307453126
3.984421681479944e-10
4.0884629015636165e-11
GMM:  0.14193782276150946
8.594538414286035e-10
9.57651735689069e-11
GMM:  0.1419380045826657
0.011027026088077818
0.0009568939894286643
8.349172658128623e-05
7.2883504307341695e-06
GMM:  287110.04954691255
0.012407625686067902
0.0013454848283025456
0.0001450754539149557
1.563264932613606e-05
GMM:  2974.8159522123196
0.0027090745108271364
0.00030145282339244517
3.3510684666282486e-05
3.7246559285719627e-06
GMM:  28.33369756313965
0.0002806872789404835
3.129099290521964e-05
3.4888195976634506e-06
3.889840982651549e-07
4.336950532035644e-08
GMM:  0.2577938797958459
2.1580652125763322e-05
2.4060556285121493e-06
2.6832571276003137e-07
GMM:  0.11358010026610232
GMM:  0.11358011110808941
GMM:  0.1135802488352532
GMM:  0.11358012234948855
GMM:  0.11

8.588276756427149e-10
9.57118828637249e-11
GMM:  0.09382335929158792
3.4116622211457646
2.2276157693934326
1.5128929088111995
1.099026589139981
0.8371793828652789
0.6587134438058726
0.5329438400588673
GMM:  1109128.8706340606
0.10158372417090078
0.010790370718861908
0.001242828379940164
0.00014434475146529735
GMM:  629.5937298128147
0.006540936071884218
0.0007235633207898218
8.028129672688067e-05
8.910450116417223e-06
GMM:  6.744032626298116
0.0004594819115517623
5.086735301595979e-05
5.62684534344271e-06
6.224444941693719e-07
6.885530723366173e-08
GMM:  0.1352464329237999
3.814688060010951e-05
4.223127135105642e-06
4.6703199885200775e-07
GMM:  0.09077989691683253
GMM:  0.09077988188069491
GMM:  0.09077988190656194
GMM:  0.09077986803573806
GMM:  0.09078003314084832
GMM:  0.09077989961963617
GMM:  0.09077988268936987
GMM:  0.09077993677513169
GMM:  0.09077996724367987
GMM:  0.09078164645877274
GMM:  0.09077988715349983
GMM:  0.09077979031382422
GMM:  0.090806391061519
GMM:  0.090780175

GMM:  0.024116421553406847
GMM:  0.024116445348545595
GMM:  0.024116405583001876
GMM:  0.024116430760803664
GMM:  0.024116420963108856
GMM:  0.024116435104167444
GMM:  0.02411642028605121
GMM:  0.02411645655043683
4.105871198589739e-10
4.2591707938299805e-11
GMM:  0.0241164622785566
8.835838727350165e-10
9.941381051703502e-11
GMM:  0.02411652012465995
0.3501582040882538
0.09799055022508041
0.02454076347804479
0.0059512572223532345
0.0014315606066617192
0.00034368190069056226
GMM:  6678.214165270045
0.07990642140342885
0.009216655990210043
0.0011134844713414083
0.00013515858449553164
1.6414475058290634e-05
GMM:  1.8757578865383255
0.007227639118362994
0.0008199200384968464
9.329057499840587e-05
1.0618692041219902e-05
GMM:  0.022569452624693787
0.0005982663616794781
6.773068859899922e-05
7.667430637070538e-06
8.680165279884022e-07
9.826699987058873e-08
GMM:  0.020827720307929093
0.00028617163065902673
3.2317246497548524e-05
3.6489186792110218e-06
4.1200324929491217e-07
GMM:  0.0218906029

2.5096057282958384e-05
2.8271289655990017e-06
3.1842645586976914e-07
3.5865214798036504e-08
GMM:  0.023029307626382167
1.3170499556736104e-05
1.4834800392549141e-06
1.6706348304751373e-07
1.8814031221836558e-08
GMM:  0.023147329509812645
6.92150081338383e-06
7.795569159085858e-07
8.778375892859458e-08
GMM:  0.02320982713261238
GMM:  0.023209823724122502
GMM:  0.02320979698343438
GMM:  0.02320984675961299
GMM:  0.02320992573957334
GMM:  0.023209849923251938
GMM:  0.02320986562431515
GMM:  0.02320991914553358
GMM:  0.023209912211734324
GMM:  0.023211742835795468
GMM:  0.02320985352363618
GMM:  0.023209756667534026
GMM:  0.02323665491790211
GMM:  0.023210104918322705
GMM:  0.023210793152843884
GMM:  0.023213167622598826
GMM:  0.023209876269312785
GMM:  0.023209713426288803
GMM:  0.023256867356293925
GMM:  0.023210303768544258
GMM:  0.023211494846589298
GMM:  0.023210116354060728
GMM:  0.023209818648713604
1.5081624837876006e-10
1.6267875935227494e-11
GMM:  0.02320982763933098
1.2704504115

5.433578671443229e-07
6.12379844611155e-08
GMM:  0.022275403915493486
2.2176514836935723e-05
2.4994857810156645e-06
2.8161905163415213e-07
GMM:  0.02260258142639758
GMM:  0.02260257054140072
GMM:  0.02260254723539054
GMM:  0.022602594688606786
GMM:  0.022602672025146764
GMM:  0.022602597472216046
GMM:  0.022602611576827553
GMM:  0.02260266827014111
GMM:  0.022602660698796948
GMM:  0.02260455874858361
GMM:  0.02260260092203137
GMM:  0.02260250523409371
GMM:  0.022630410401217327
GMM:  0.022602853971000147
GMM:  0.02260357245114289
GMM:  0.022606040124645593
GMM:  0.022602624603916946
GMM:  0.02260246371034011
GMM:  0.02265143340861131
GMM:  0.022603055380858786
GMM:  0.022604301549339706
GMM:  0.02260287473137565
GMM:  0.02260256542067013
1.517630465741604e-10
1.638511548662791e-11
GMM:  0.02260257420596512
GMM:  0.022602564184217785
GMM:  0.02260258884406166
GMM:  0.02260254916995788
GMM:  0.022602574840406967
GMM:  0.02260256393282073
GMM:  0.022602578816545177
GMM:  0.022602563729587

C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in multiply


7.59562733664141
7.594532763887558
7.594529784967621
7.589764835086132
7.205801878508822
GMM:  16078548137.706453
9.060866627934264
9.06085666092596
9.060785450986288
9.045848331361391
8.87074297370664
8.765828109975878
8.695621081778404
8.693144961687764
8.693144501279596
8.693144501202331
8.693144501016945
8.693143395859208
8.686904039371893
8.575716098808078
8.575288016783519
8.4755163718526
8.232235256156878
8.15928917716914
8.105337728233238
8.056584319184344
7.982416340997707
7.981052654894537
7.980832074679665
7.876580857453282
7.82801577928798
7.826820775341133
GMM:  169731938.64510095
7.144360130529659
7.059186192212008
6.968523732122122
6.73143920279726
6.500158104049405
6.415659192733889
GMM:  35432133.45051902
6.0805735438323
5.772849863524806
5.683217136745278
5.638530654205612
4.898026625541027
GMM:  21907850.181707498
4.8928709306423315
4.89269237471159
4.8924204021291855
4.858198507998176
GMM:  10292298.470739298
4.664783942881874
4.630447264432837
4.56805630211865
4.56

In [63]:
a1,a2,a3,a4,a5,b1,b2,s1,s2,s3,s4,s5,s6,g1,g2,g3,g4,g5,g6,theta,L11,L21,L22,L31,L32,L41,L42,L51,L52,lx1,lx2,lambda_price = results.x

results

     fun: 67806.26867814054
     jac: array([-1.49018180e+00,  1.67247842e+00,  6.85921837e+00,  1.86057267e+00,
        2.78955564e+00,  6.65247803e+00,  6.12696321e+00,  1.34802727e+02,
        2.08841973e+00, -4.32612182e+00,  1.88752592e+03,  1.91279010e+01,
        6.79162398e+01,  2.35294131e+02,  3.69533270e+00, -7.09061539e+00,
        3.31360762e+03,  3.27096577e+01,  1.17370770e+02,  2.06716853e+01,
       -3.09670537e-01,  2.78181356e-01, -4.32728588e-01,  1.28959480e+00,
       -1.43989586e+00,  3.19861113e-01, -4.21508213e-01,  6.04930573e-01,
       -4.43541043e-01,  2.18346837e+00,  2.59007136e+00,  6.96203336e+00])
 message: 'Optimization terminated successfully.'
    nfev: 980
     nit: 24
    njev: 24
  status: 0
 success: True
       x: array([ 6.80758953e-01, -5.96701125e-01, -6.80484152e-01, -9.25745427e-01,
       -1.20765814e+00,  7.27950205e-01,  4.37131645e-01, -6.17055874e-01,
       -1.20008299e+00, -4.30807453e-01,  9.35227977e-03,  1.29541219e-01,
       -5

In [52]:
results_GMM=np.array([ 6.80758953e-01, -5.96701125e-01, -6.80484152e-01, -9.25745427e-01,\
                      -1.20765814e+00,  7.27950205e-01,  4.37131645e-01, -6.17055874e-01,\
                      -1.20008299e+00, -4.30807453e-01,  9.35227977e-03,  1.29541219e-01,\
                      -5.66878115e-02,  4.67976536e-01,  5.53758549e-01,  5.93789141e-02,\
                      -4.88445455e-03,  1.27182237e-01,  3.55481346e-02, -5.42338434e+00,\
                      1.67181676e-01,  2.73608683e-01,  7.83595225e-02,  2.69669025e-01,\
                      1.23692722e-01,  3.12445637e-01,  1.00367643e-01,  3.50381323e-01,\
                      1.70956266e-01,  2.03323646e-02,  1.20852989e-01,  4.54971109e-01])

(a1,a2,a3,a4,a5,b1,b2,s1,s2,s3,s4,s5,s6,g1,g2,g3,g4,g5,g6,\
 theta,L11,L21,L22,L31,L32,L41,L42,L51,L52,lx1,lx2,lambda_price)=results_GMM

In [53]:
# Calculating the elasticities

alpha_def=np.array([a1, a2, a3, a4, a5])
beta_def=np.array([b1, b2])
sigma_def=np.array([s1, s2, s3, s4, s5, s6])
gamma_def=np.array([g1, g2, g3, g4, g5, g6])
L_matrix_def=np.array([[L11, 0], [L21, L22],  [L31, L32],  \
                   [L41,  L42],  [L51, L52]])
lambda_p_s_def=np.array([lx1, lx2])
theta_def=theta
lambda_price_def=lambda_price

In [54]:
delta_def=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, price_est, storeweek)

In [55]:
mu_def=mu_fun(promosize, price_est, brands, L_matrix_def, lambda_p_s_def,lambda_price_def, normal_beta_draws, normal_theta_draws, normal_w_draws)

In [56]:
#calculating the equilibrium market shares

th2=(L_matrix_def, lambda_p_s_def,lambda_price_def)
data=(promosize, demog, brands, price_est, storeweek,share)
simpack=(normal_beta_draws, normal_theta_draws, normal_w_draws)
d_test, est_sh_test =est_ms_cm(delta_def, th2, data, simpack)

In [57]:
# Price differentials [I NEED TO CREATE PRICES CHANGES FOR 64 AND 128 OZ SEPARATELY]
h=1e-5
P_Tide64_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['Tide']*datapack_GMM['64 OZ']), ndmin=2).T
P_cheer64_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['cheer']*datapack_GMM['64 OZ']), ndmin=2).T
P_wisk64_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['wisk']*datapack_GMM['64 OZ']), ndmin=2).T
P_all64_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['all_det']*datapack_GMM['64 OZ']), ndmin=2).T
P_surf64_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['surf']*datapack_GMM['64 OZ']), ndmin=2).T

P_Tide128_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['Tide']*datapack_GMM['128 OZ']), ndmin=2).T
P_cheer128_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['cheer']*datapack_GMM['128 OZ']), ndmin=2).T
P_wisk128_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['wisk']*datapack_GMM['128 OZ']), ndmin=2).T
P_all128_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['all_det']*datapack_GMM['128 OZ']), ndmin=2).T
P_surf128_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['surf']*datapack_GMM['128 OZ']), ndmin=2).T

In [58]:
#delta function with price differentials (surf 128 OZ is not included in the paper)

d_Tide64=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_Tide64_diff, storeweek)
d_cheer64=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_cheer64_diff, storeweek)
d_wisk64=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_wisk64_diff, storeweek)
d_all64=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_all64_diff, storeweek)
d_surf64=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_surf64_diff, storeweek)

d_Tide128=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_Tide128_diff, storeweek)
d_cheer128=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_cheer128_diff, storeweek)
d_wisk128=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_wisk128_diff, storeweek)
d_all128=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_all128_diff, storeweek)


In [59]:
P_Tide64_diff.shape

(35360, 1)

In [60]:
# calculating the estimated market shares upon a change in price

data_Tide64=(promosize, demog, brands,P_Tide64_diff , storeweek,share)
dt, shares_Tide64=est_ms_cm(d_Tide64, th2, data_Tide64, simpack)

data_cheer64=(promosize, demog, brands, P_cheer64_diff, storeweek,share)
dc, shares_cheer64=est_ms_cm(d_cheer64, th2, data_cheer64, simpack)

data_wisk64=(promosize, demog, brands, P_wisk64_diff, storeweek,share)
dw, shares_wisk64=est_ms_cm(d_wisk64, th2, data_wisk64, simpack)

data_all64=(promosize, demog, brands, P_all64_diff, storeweek,share)
da, shares_all64=est_ms_cm(d_all64, th2, data_all64, simpack)

data_surf64=(promosize, demog, brands, P_surf64_diff, storeweek,share)
ds, shares_surf64=est_ms_cm(d_surf64, th2, data_surf64, simpack)

data_Tide128=(promosize, demog, brands, P_Tide128_diff, storeweek,share)
dt, shares_Tide128=est_ms_cm(d_Tide128, th2, data_Tide128, simpack)

data_cheer128=(promosize, demog, brands, P_cheer128_diff, storeweek,share)
dc, shares_cheer128=est_ms_cm(d_cheer128, th2, data_cheer128, simpack)

data_wisk128=(promosize, demog, brands, P_wisk128_diff, storeweek,share)
dw, shares_wisk128=est_ms_cm(d_wisk128, th2, data_wisk128, simpack)

data_all128=(promosize, demog, brands, P_all128_diff, storeweek,share)
da, shares_all128=est_ms_cm(d_all128, th2, data_all128, simpack)


In [61]:
def filter_1(dif,dataset, filter1, value1, filter2, value2,col_groupby):
    data_i=dataset.loc[lambda di: dataset[filter1]==value1,:]
    data_m=data_i.loc[lambda df: data_i[filter2]==value2,:]
    data_f=data_m.groupby([col_groupby]).sum()
    data_f.iloc[:,11]=(data_f.iloc[:,11]-data_f.iloc[:,10])/dif
    data_f.iloc[:,12]=(data_f.iloc[:,12]-data_f.iloc[:,10])/dif
    data_f.iloc[:,13]=(data_f.iloc[:,13]-data_f.iloc[:,10])/dif
    data_f.iloc[:,14]=(data_f.iloc[:,14]-data_f.iloc[:,10])/dif
    data_f.iloc[:,15]=(data_f.iloc[:,15]-data_f.iloc[:,10])/dif
    data_f.iloc[:,16]=(data_f.iloc[:,16]-data_f.iloc[:,10])/dif
    data_f.iloc[:,17]=(data_f.iloc[:,17]-data_f.iloc[:,10])/dif
    data_f.iloc[:,18]=(data_f.iloc[:,18]-data_f.iloc[:,10])/dif
    data_f.iloc[:,19]=(data_f.iloc[:,19]-data_f.iloc[:,10])/dif
    return data_f

In [62]:
# generating the filtered data, each row in the 'share_x' columns show the effect in the 'brand' and 'size' market share 
# of a change in the price of '_x'

zone_week=np.array(datapack_GMM['zone'].apply(lambda x: x*1000)+datapack_GMM['week'], ndmin=2).T
week=np.array(datapack_GMM['week'], ndmin=2).T
totcount = np.array(datapack_GMM['totcount'], ndmin=2).T

q_diff = np.hstack((zone_week, week, storeweek, promosize, brands, totcount, est_sh_test, \
                    shares_Tide64, shares_Tide128, shares_cheer64, shares_cheer128, \
                    shares_wisk64, shares_wisk128, shares_all64, shares_all128, shares_surf64))

q_diff=pd.DataFrame(q_diff)

q_diff.columns = ['zone-week', 'week', 'store-week', 'promo', '64_OZ', 'Tide','cheer','wisk','all_det','surf'\
                  , 'totcount', 'est_sh_eq', 'shares_pT64', 'shares_pT128', 'shares_pc64'\
                  , 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64']

h=1e-5

sh_Tide_64=filter_1(h, q_diff, 'Tide', 1, '64_OZ', 1, 'store-week')
sh_Tide_64['store-week']=sh_Tide_64.index.values

sh_Tide_128=filter_1(h, q_diff, 'Tide', 1, '64_OZ', 0, 'store-week')
sh_Tide_128['store-week']=sh_Tide_128.index.values

sh_cheer_64=filter_1(h, q_diff, 'cheer', 1, '64_OZ', 1, 'store-week')
sh_cheer_64['store-week']=sh_cheer_64.index.values

sh_cheer_128=filter_1(h, q_diff, 'cheer', 1, '64_OZ', 0, 'store-week')
sh_cheer_128['store-week']=sh_cheer_128.index.values

sh_wisk_64=filter_1(h, q_diff, 'wisk', 1, '64_OZ', 1, 'store-week')
sh_wisk_64['store-week']=sh_wisk_64.index.values

sh_wisk_128=filter_1(h, q_diff, 'wisk', 1, '64_OZ', 0, 'store-week')
sh_wisk_128['store-week']=sh_wisk_128.index.values

sh_all_64=filter_1(h, q_diff, 'all_det', 1, '64_OZ', 1, 'store-week')
sh_all_64['store-week']=sh_all_64.index.values

sh_all_128=filter_1(h, q_diff, 'all_det', 1, '64_OZ', 0, 'store-week')
sh_all_128['store-week']=sh_all_128.index.values

sh_surf_64=filter_1(h, q_diff, 'surf', 1, '64_OZ', 1, 'store-week')
sh_surf_64['store-week']=sh_surf_64.index.values

In [63]:
def weighted_prices(data, filter1, valuef1, filter2, valuef2, price1, price2, weight_col,groupby_col):
    prices_ini=data.loc[lambda pcs: data[filter1]==valuef1,:]
    prices_med=prices_ini.loc[lambda pcm: prices_ini[filter2]==valuef2,:]
    prices_med['weighted_price'] = prices_med[price1]*prices_med[weight_col]
    prices_med['weighted_wprice'] = prices_med[price2]*prices_med[weight_col]
    prices_fin = prices_med.groupby([groupby_col]).sum()
    prices_fin['weighted_price'] = prices_fin['weighted_price']/prices_fin[weight_col]
    prices_fin['weighted_wprice'] = prices_fin['weighted_wprice']/prices_fin[weight_col]
#    prices_fin[groupby_col] = prices_fin[groupby_col]/prices_fin[filter1]
    return prices_fin

In [64]:
#It could be possible that a product had not been sold in a store-week, 
#so for pricing it is necessary to exactly math the cuantities with the respectives products

#calculating weighted average store-week price and weighted average store-week wholesale price. This data also includes the
#Q total per brand, size, and store-week

move=np.array(datapack_GMM[['move']])

price_data=np.hstack((zone_week, storeweek, week, promosize, move, brands, price_est, wholeprice))
price_data=pd.DataFrame(price_data)

price_data.columns = ['zone-week', 'store-week', 'week', 'promo', '64_OZ', 'move', 'Tide','cheer','wisk','all_det',\
                      'surf', 'price_est', 'wholeprice']

Tide64_prices = weighted_prices(price_data, 'Tide', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'store-week')
Tide64_prices['store-week']=Tide64_prices.index.values

Tide128_prices = weighted_prices(price_data, 'Tide', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'store-week')
Tide128_prices['store-week']=Tide128_prices.index.values

cheer64_prices = weighted_prices(price_data, 'cheer', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'store-week')
cheer64_prices['store-week']=cheer64_prices.index.values

cheer128_prices = weighted_prices(price_data, 'cheer', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'store-week')
cheer128_prices['store-week']=cheer128_prices.index.values

wisk64_prices = weighted_prices(price_data, 'wisk', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'store-week')
wisk64_prices['store-week']=wisk64_prices.index.values

wisk128_prices = weighted_prices(price_data, 'wisk', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'store-week')
wisk128_prices['store-week']=wisk128_prices.index.values

all64_prices = weighted_prices(price_data, 'all_det', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'store-week')
all64_prices['store-week']=all64_prices.index.values

all128_prices = weighted_prices(price_data, 'all_det', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'store-week')
all128_prices['store-week']=all128_prices.index.values

surf64_prices = weighted_prices(price_data, 'surf', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'store-week')
surf64_prices['store-week']=surf64_prices.index.values

C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [65]:
def mapping(data1, data2, common, col1, col2, col3, col4, col5, col6, col7, col8, col9):
    m1=data1[common].map(data2.set_index(common)[col1])
    m2=data1[common].map(data2.set_index(common)[col2])
    m3=data1[common].map(data2.set_index(common)[col3])
    m4=data1[common].map(data2.set_index(common)[col4])
    m5=data1[common].map(data2.set_index(common)[col5])
    m6=data1[common].map(data2.set_index(common)[col6])
    m7=data1[common].map(data2.set_index(common)[col7])
    m8=data1[common].map(data2.set_index(common)[col8])
    m9=data1[common].map(data2.set_index(common)[col9])
    mapping=pd.concat([m1, m2, m3, m4, m5, m6, m7, m8, m9], axis=1)
    mapping.columns=['shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64']
    return mapping

In [93]:
#calculating equilibrium prices on store configuration

market2=datapack_GMM['store-week'].drop_duplicates()

#lndfbrand_d2 = pd.concat([a,b,c], axis=1), or 
#datapack_GMM.loc[:,'income']=datapack_GMM['store'].map(zones.set_index('store')['income']).fillna(0)

wholesale_prices_s=pd.DataFrame(datapack_GMM['store-week'].drop_duplicates())
wholesale_prices_s['t64']=wholesale_prices_s['store-week'].map(Tide64_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['t128']=wholesale_prices_s['store-week'].map(Tide128_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['c64']=wholesale_prices_s['store-week'].map(cheer64_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['c128']=wholesale_prices_s['store-week'].map(cheer128_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['w64']=wholesale_prices_s['store-week'].map(wisk64_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['w128']=wholesale_prices_s['store-week'].map(wisk128_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['a64']=wholesale_prices_s['store-week'].map(all64_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['a128']=wholesale_prices_s['store-week'].map(all128_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['s64']=wholesale_prices_s['store-week'].map(surf64_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s=wholesale_prices_s.drop(columns=['store-week'], axis=1)

elasticities_s=pd.DataFrame(datapack_GMM['store-week'].drop_duplicates())
es_t64=mapping(elasticities_s,sh_Tide_64,'store-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
es_t128=mapping(elasticities_s,sh_Tide_128,'store-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
es_c64=mapping(elasticities_s,sh_cheer_64,'store-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
es_c128=mapping(elasticities_s,sh_cheer_128,'store-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
es_w64=mapping(elasticities_s,sh_wisk_64,'store-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
es_w128=mapping(elasticities_s,sh_wisk_128,'store-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
es_a64=mapping(elasticities_s,sh_all_64,'store-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
es_a128=mapping(elasticities_s,sh_all_128,'store-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
es_s64=mapping(elasticities_s,sh_surf_64,'store-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')

elasticities_s=pd.concat([elasticities_s, es_t64,es_t128,es_c64, es_c128, es_w64,es_w128, es_a64,es_a128, es_s64], axis=1)
elasticities_s=elasticities_s.drop(columns=['store-week'], axis=1)

Quants_s=pd.DataFrame(datapack_GMM['store-week'].drop_duplicates())
Quants_s['t64']=Quants_s['store-week'].map(Tide64_prices.set_index('store-week')['move'])
Quants_s['t128']=Quants_s['store-week'].map(Tide128_prices.set_index('store-week')['move'])
Quants_s['c64']=Quants_s['store-week'].map(cheer64_prices.set_index('store-week')['move'])
Quants_s['c128']=Quants_s['store-week'].map(cheer128_prices.set_index('store-week')['move'])
Quants_s['w64']=Quants_s['store-week'].map(wisk64_prices.set_index('store-week')['move'])
Quants_s['w128']=Quants_s['store-week'].map(wisk128_prices.set_index('store-week')['move'])
Quants_s['a64']=Quants_s['store-week'].map(all64_prices.set_index('store-week')['move'])
Quants_s['a128']=Quants_s['store-week'].map(all128_prices.set_index('store-week')['move'])
Quants_s['s64']=Quants_s['store-week'].map(surf64_prices.set_index('store-week')['move'])
Quants_s=Quants_s.drop(columns=['store-week'], axis=1)

opt_price_ST = pd.concat([market2, wholesale_prices_s, elasticities_s, Quants_s], axis=1, sort=False)

opt_pricesST_matrix=np.zeros((market2.size,9))

for i in range(market2.shape[0]):
    omega=np.vstack([[opt_price_ST.iloc[i,10:19]],[opt_price_ST.iloc[i,19:28]],[opt_price_ST.iloc[i,28:37]]\
                     ,[opt_price_ST.iloc[i,37:46]],[opt_price_ST.iloc[i,46:55]],[opt_price_ST.iloc[i,55:64]]\
                     ,[opt_price_ST.iloc[i,64:73]],[opt_price_ST.iloc[i,73:82]],[opt_price_ST.iloc[i,82:91]]])
    omega_inv=lin.inv(omega)
    whpr=np.reshape(np.array(opt_price_ST.iloc[i,1:10]),(9,1)) - omega_inv@np.reshape(np.array(opt_price_ST.iloc[i,91:100]),(9,1))
    opt_pricesST_matrix[i,:]=whpr.T
    i+=1

market2=np.array(market2,ndmin=2).T

opt_pricesST_matrix=np.hstack((market2, opt_pricesST_matrix))

opt_pricesST=pd.DataFrame(opt_pricesST_matrix)
opt_pricesST.columns=['store-week','t64_s','t128_s', 'c64_s', 'c128_s', 'w64_s', 'w128_s', 'a64_s', 'a128_s', 's64_s']

In [67]:
market2.shape

(3905, 1)

### Zone configuration

What did I do?: i) to calculate dq/dp for each zone-week, using the weighted average per store shares (wieghted by totalcount), ii) to calculate weighted average prices and wholesale prices for zone-week, and, iii) using dp/dq, quantities and wholesale prices, calculate the equilibrium price for this configuration.

In [68]:
def filter_2(dif,dataset, filter1, value1, filter2, value2,col_groupby1, col_groupby2, weight):
    data_i=dataset.loc[lambda di: dataset[filter1]==value1,:]
    data_m=data_i.loc[lambda df: data_i[filter2]==value2,:]
    data_m=data_m.groupby([col_groupby1]).sum()
    data_m.iloc[:,11]=data_m.iloc[:,11]*data_m[weight]
    data_m.iloc[:,12]=data_m.iloc[:,12]*data_m[weight]
    data_m.iloc[:,13]=data_m.iloc[:,13]*data_m[weight]
    data_m.iloc[:,14]=data_m.iloc[:,14]*data_m[weight]
    data_m.iloc[:,15]=data_m.iloc[:,15]*data_m[weight]
    data_m.iloc[:,16]=data_m.iloc[:,16]*data_m[weight]
    data_m.iloc[:,17]=data_m.iloc[:,17]*data_m[weight]
    data_m.iloc[:,18]=data_m.iloc[:,18]*data_m[weight]
    data_m.iloc[:,19]=data_m.iloc[:,19]*data_m[weight]
    data_f=data_m.groupby([col_groupby2]).sum()
    data_f.iloc[:,10]=data_f.iloc[:,10]*data_f[weight]
    data_f.iloc[:,11]=data_f.iloc[:,11]*data_f[weight]
    data_f.iloc[:,12]=data_f.iloc[:,12]*data_f[weight]
    data_f.iloc[:,13]=data_f.iloc[:,13]*data_f[weight]
    data_f.iloc[:,14]=data_f.iloc[:,14]*data_f[weight]
    data_f.iloc[:,15]=data_f.iloc[:,15]*data_f[weight]
    data_f.iloc[:,16]=data_f.iloc[:,16]*data_f[weight]
    data_f.iloc[:,17]=data_f.iloc[:,17]*data_f[weight]
    data_f.iloc[:,18]=data_f.iloc[:,18]*data_f[weight]
#    data_f[col_groupby1] = data_f[col_groupby1]/data_f[filter1]
#    data_f[col_groupby2] = data_f[col_groupby2]/data_f[filter1]
    data_f.iloc[:,10]=(data_f.iloc[:,10]-data_f.iloc[:,9])/dif
    data_f.iloc[:,11]=(data_f.iloc[:,11]-data_f.iloc[:,9])/dif
    data_f.iloc[:,12]=(data_f.iloc[:,12]-data_f.iloc[:,9])/dif
    data_f.iloc[:,13]=(data_f.iloc[:,13]-data_f.iloc[:,9])/dif
    data_f.iloc[:,14]=(data_f.iloc[:,14]-data_f.iloc[:,9])/dif
    data_f.iloc[:,15]=(data_f.iloc[:,15]-data_f.iloc[:,9])/dif
    data_f.iloc[:,16]=(data_f.iloc[:,16]-data_f.iloc[:,9])/dif
    data_f.iloc[:,17]=(data_f.iloc[:,17]-data_f.iloc[:,9])/dif
    data_f.iloc[:,18]=(data_f.iloc[:,18]-data_f.iloc[:,9])/dif
    return data_f

In [69]:
sh_T64_zone=filter_2(h, q_diff, 'Tide', 1, '64_OZ', 1, 'store-week', 'zone-week', 'totcount')
sh_T64_zone['zone-week']=sh_T64_zone.index.values

sh_T128_zone=filter_2(h, q_diff, 'Tide', 1, '64_OZ', 0, 'store-week', 'zone-week', 'totcount')
sh_T128_zone['zone-week']=sh_T64_zone.index.values

sh_c64_zone=filter_2(h, q_diff, 'cheer', 1, '64_OZ', 1, 'store-week', 'zone-week', 'totcount')
sh_c64_zone['zone-week']=sh_c64_zone.index.values

sh_c128_zone=filter_2(h, q_diff, 'cheer', 1, '64_OZ', 0, 'store-week', 'zone-week', 'totcount')
sh_c128_zone['zone-week']=sh_c128_zone.index.values

sh_w64_zone=filter_2(h, q_diff, 'wisk', 1, '64_OZ', 1, 'store-week', 'zone-week', 'totcount')
sh_w64_zone['zone-week']=sh_w64_zone.index.values

sh_w128_zone=filter_2(h, q_diff, 'wisk', 1, '64_OZ', 0, 'store-week', 'zone-week', 'totcount')
sh_w128_zone['zone-week']=sh_w128_zone.index.values

sh_a64_zone=filter_2(h, q_diff, 'all_det', 1, '64_OZ', 1, 'store-week', 'zone-week', 'totcount')
sh_a64_zone['zone-week']=sh_a64_zone.index.values

sh_a128_zone=filter_2(h, q_diff, 'all_det', 1, '64_OZ', 0, 'store-week', 'zone-week', 'totcount')
sh_a128_zone['zone-week']=sh_a128_zone.index.values

sh_s64_zone=filter_2(h, q_diff, 'surf', 1, '64_OZ', 1, 'store-week', 'zone-week', 'totcount')
sh_s64_zone['zone-week']=sh_s64_zone.index.values

In [70]:
T64_prices_zone = weighted_prices(price_data, 'Tide', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'zone-week')
T64_prices_zone['zone-week']=T64_prices_zone.index.values

T128_prices_zone = weighted_prices(price_data, 'Tide', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'zone-week')
T128_prices_zone['zone-week']=T128_prices_zone.index.values

c64_prices_zone = weighted_prices(price_data, 'cheer', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'zone-week')
c64_prices_zone['zone-week']=c64_prices_zone.index.values

c128_prices_zone = weighted_prices(price_data, 'cheer', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'zone-week')
c128_prices_zone['zone-week']=c128_prices_zone.index.values

w64_prices_zone = weighted_prices(price_data, 'wisk', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'zone-week')
w64_prices_zone['zone-week']=w64_prices_zone.index.values

w128_prices_zone = weighted_prices(price_data, 'wisk', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'zone-week')
w128_prices_zone['zone-week']=w128_prices_zone.index.values

a64_prices_zone = weighted_prices(price_data, 'all_det', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'zone-week')
a64_prices_zone['zone-week']=a64_prices_zone.index.values

a128_prices_zone = weighted_prices(price_data, 'all_det', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'zone-week')
a128_prices_zone['zone-week']=a128_prices_zone.index.values

s64_prices_zone = weighted_prices(price_data, 'surf', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'zone-week')
s64_prices_zone['zone-week']=s64_prices_zone.index.values

C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [71]:
#sh_T64_zone

In [79]:
#calculating equilibrium prices on zone configuration

zone_week=pd.DataFrame(zone_week)
market3=zone_week.drop_duplicates()

#lndfbrand_d2 = pd.concat([a,b,c], axis=1), or 
#lndf_ok.loc[:,'income']=lndf_ok['zone'].map(zones.set_index('zone')['income']).fillna(0)


wholesale_prices_z=zone_week.drop_duplicates()
wholesale_prices_z.columns=['zone-week']
wholesale_prices_z['t64']=wholesale_prices_z['zone-week'].map(T64_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['t128']=wholesale_prices_z['zone-week'].map(T128_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['c64']=wholesale_prices_z['zone-week'].map(c64_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['c128']=wholesale_prices_z['zone-week'].map(c128_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['w64']=wholesale_prices_z['zone-week'].map(w64_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['w128']=wholesale_prices_z['zone-week'].map(w128_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['a64']=wholesale_prices_z['zone-week'].map(a64_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['a128']=wholesale_prices_z['zone-week'].map(a128_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['s64']=wholesale_prices_z['zone-week'].map(s64_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z=wholesale_prices_z.drop(columns=['zone-week'], axis=1)

elasticities_z=zone_week.drop_duplicates()
elasticities_z.columns=['zone-week']
ez_t64=mapping(elasticities_z,sh_T64_zone,'zone-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ez_t128=mapping(elasticities_z,sh_T128_zone,'zone-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ez_c64=mapping(elasticities_z,sh_c64_zone,'zone-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ez_c128=mapping(elasticities_z,sh_c128_zone,'zone-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ez_w64=mapping(elasticities_z,sh_w64_zone,'zone-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ez_w128=mapping(elasticities_z,sh_w128_zone,'zone-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ez_a64=mapping(elasticities_z,sh_a64_zone,'zone-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ez_a128=mapping(elasticities_z,sh_a128_zone,'zone-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ez_s64=mapping(elasticities_z,sh_s64_zone,'zone-week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')

elasticities_z=pd.concat([elasticities_z, ez_t64,ez_t128,ez_c64, ez_c128, ez_w64,ez_w128, ez_a64,ez_a128, ez_s64], axis=1)
elasticities_z=elasticities_z.drop(columns=['zone-week'], axis=1)

Quants_z=zone_week.drop_duplicates()
Quants_z.columns=['zone-week']                                           
Quants_z['t64']=Quants_z['zone-week'].map(T64_prices_zone.set_index('zone-week')['move'])
Quants_z['t128']=Quants_z['zone-week'].map(T128_prices_zone.set_index('zone-week')['move'])
Quants_z['c64']=Quants_z['zone-week'].map(c64_prices_zone.set_index('zone-week')['move'])
Quants_z['c128']=Quants_z['zone-week'].map(c128_prices_zone.set_index('zone-week')['move'])
Quants_z['w64']=Quants_z['zone-week'].map(w64_prices_zone.set_index('zone-week')['move'])
Quants_z['w128']=Quants_z['zone-week'].map(w128_prices_zone.set_index('zone-week')['move'])
Quants_z['a64']=Quants_z['zone-week'].map(a64_prices_zone.set_index('zone-week')['move'])
Quants_z['a128']=Quants_z['zone-week'].map(a128_prices_zone.set_index('zone-week')['move'])
Quants_z['s64']=Quants_z['zone-week'].map(s64_prices_zone.set_index('zone-week')['move'])
Quants_z=Quants_z.drop(columns=['zone-week'], axis=1)
                                           
opt_price_ZN = np.hstack((market3, wholesale_prices_z, elasticities_z, Quants_z))

opt_price_ZN = pd.DataFrame(opt_price_ZN)

opt_pricesZN_matrix=np.zeros((market3.size,9))
i=0
for i in range(market3.shape[0]):
    omegaZN=np.vstack([[opt_price_ZN.iloc[i,10:19]],[opt_price_ZN.iloc[i,19:28]],[opt_price_ZN.iloc[i,28:37]]\
                   ,[opt_price_ZN.iloc[i,37:46]],[opt_price_ZN.iloc[i,46:55]],[opt_price_ZN.iloc[i,55:64]]\
                   ,[opt_price_ZN.iloc[i,64:73]],[opt_price_ZN.iloc[i,73:82]],[opt_price_ZN.iloc[i,82:91]]])
    omegaZN_inv=lin.inv(omegaZN)
    whprZN=np.reshape(np.array(opt_price_ZN.iloc[i,1:10]),(9,1)) - \
    omegaZN_inv@np.reshape(np.array(opt_price_ZN.iloc[i,91:100]),(9,1))
    opt_pricesZN_matrix[i,:]=whprZN.T
    i+=1

market3=np.array(market3,ndmin=2)

opt_pricesZN_matrix=np.hstack((market3, opt_pricesZN_matrix))
                                           
opt_pricesZN=pd.DataFrame(opt_pricesZN_matrix)
opt_pricesZN.columns=['zone-week','t64_z','t128_z', 'c64_z', 'c128_z', 'w64_z', 'w128_z', 'a64_z', 'a128_z', 's64_z']

C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

### Chain configuration

What did I do?: i) to calculate dq/dp for each week, using the weighted average per store shares (wieghted by totalcount), ii) to calculate weighted average prices and wholesale prices for  each week, and, iii) using dp/dq, quantities and wholesale prices, calculate the equilibrium price for this configuration.

In [80]:
sh_T64_chain=filter_2(h, q_diff, 'Tide', 1, '64_OZ', 1, 'store-week', 'week', 'totcount')
sh_T64_chain['week']=sh_T64_chain.index.values

sh_T128_chain=filter_2(h, q_diff, 'Tide', 1, '64_OZ', 0, 'store-week', 'week', 'totcount')
sh_T128_chain['week']=sh_T128_chain.index.values

sh_c64_chain=filter_2(h, q_diff, 'cheer', 1, '64_OZ', 1, 'store-week', 'week', 'totcount')
sh_c64_chain['week']=sh_c64_chain.index.values

sh_c128_chain=filter_2(h, q_diff, 'cheer', 1, '64_OZ', 0, 'store-week', 'week', 'totcount')
sh_c128_chain['week']=sh_c128_chain.index.values

sh_w64_chain=filter_2(h, q_diff, 'wisk', 1, '64_OZ', 1, 'store-week', 'week', 'totcount')
sh_w64_chain['week']=sh_w64_chain.index.values

sh_w128_chain=filter_2(h, q_diff, 'wisk', 1, '64_OZ', 0, 'store-week', 'week', 'totcount')
sh_w128_chain['week']=sh_w128_chain.index.values

sh_a64_chain=filter_2(h, q_diff, 'all_det', 1, '64_OZ', 1, 'store-week', 'week', 'totcount')
sh_a64_chain['week']=sh_a64_chain.index.values

sh_a128_chain=filter_2(h, q_diff, 'all_det', 1, '64_OZ', 0, 'store-week', 'week', 'totcount')
sh_a128_chain['week']=sh_a128_chain.index.values

sh_s64_chain=filter_2(h, q_diff, 'surf', 1, '64_OZ', 1, 'store-week', 'week', 'totcount')
sh_s64_chain['week']=sh_s64_chain.index.values

In [81]:
T64_prices_chain = weighted_prices(price_data, 'Tide', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'week')
T64_prices_chain['week']=T64_prices_chain.index.values

T128_prices_chain = weighted_prices(price_data, 'Tide', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'week')
T128_prices_chain['week']=T128_prices_chain.index.values

c64_prices_chain = weighted_prices(price_data, 'cheer', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'week')
c64_prices_chain['week']=c64_prices_chain.index.values

c128_prices_chain = weighted_prices(price_data, 'cheer', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'week')
c128_prices_chain['week']=c128_prices_chain.index.values

w64_prices_chain = weighted_prices(price_data, 'wisk', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'week')
w64_prices_chain['week']=w64_prices_chain.index.values

w128_prices_chain = weighted_prices(price_data, 'wisk', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'week')
w128_prices_chain['week']=w128_prices_chain.index.values

a64_prices_chain = weighted_prices(price_data, 'all_det', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'week')
a64_prices_chain['week']=a64_prices_chain.index.values

a128_prices_chain = weighted_prices(price_data, 'all_det', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'week')
a128_prices_chain['week']=a128_prices_chain.index.values

s64_prices_chain = weighted_prices(price_data, 'surf', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'week')
s64_prices_chain['week']=s64_prices_chain.index.values

C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [82]:
#sh_s64_chain

In [101]:
#calculating equilibrium prices on chain configuration

week=pd.DataFrame(week)
market4=week.drop_duplicates()

#lndfbrand_d2 = pd.concat([a,b,c], axis=1), or 
#lndf_ok.loc[:,'income']=lndf_ok['store'].map(chains.set_index('store')['income']).fillna(0)

wholesale_prices_ch=week.drop_duplicates()
wholesale_prices_ch.columns=['week']
wholesale_prices_ch['t64']=wholesale_prices_ch['week'].map(T64_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_ch['t128']=wholesale_prices_ch['week'].map(T128_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_ch['c64']=wholesale_prices_ch['week'].map(c64_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_ch['c128']=wholesale_prices_ch['week'].map(c128_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_ch['w64']=wholesale_prices_ch['week'].map(w64_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_ch['w128']=wholesale_prices_ch['week'].map(w128_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_ch['a64']=wholesale_prices_ch['week'].map(a64_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_ch['a128']=wholesale_prices_ch['week'].map(a128_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_ch['s64']=wholesale_prices_ch['week'].map(s64_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_ch=wholesale_prices_ch.drop(columns=['week'], axis=1)

elasticities_ch=week.drop_duplicates()
elasticities_ch.columns=['week']
ech_t64=mapping(elasticities_ch,sh_T64_chain,'week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ech_t128=mapping(elasticities_ch,sh_T128_chain,'week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ech_c64=mapping(elasticities_ch,sh_c64_chain,'week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ech_c128=mapping(elasticities_ch,sh_c128_chain,'week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ech_w64=mapping(elasticities_ch,sh_w64_chain,'week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ech_w128=mapping(elasticities_ch,sh_w128_chain,'week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ech_a64=mapping(elasticities_ch,sh_a64_chain,'week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ech_a128=mapping(elasticities_ch,sh_a128_chain,'week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')
ech_s64=mapping(elasticities_ch,sh_s64_chain,'week','shares_pT64', 'shares_pT128', 'shares_pc64', 'shares_pc128', 'shares_pw64', 'shares_pw128', 'shares_pa64', 'shares_pa128', 'shares_ps64')

elasticities_ch=pd.concat([elasticities_ch, ech_t64,ech_t128,ech_c64, ech_c128, ech_w64,ech_w128, ech_a64,ech_a128, ech_s64], axis=1)
elasticities_ch=elasticities_ch.drop(columns=['week'], axis=1)

Quants_ch=pd.DataFrame(datapack_GMM['week'].drop_duplicates())
Quants_ch['t64']=Quants_ch['week'].map(T64_prices_chain.set_index('week')['move'])
Quants_ch['t128']=Quants_ch['week'].map(T128_prices_chain.set_index('week')['move'])
Quants_ch['c64']=Quants_ch['week'].map(c64_prices_chain.set_index('week')['move'])
Quants_ch['c128']=Quants_ch['week'].map(c128_prices_chain.set_index('week')['move'])
Quants_ch['w64']=Quants_ch['week'].map(w64_prices_chain.set_index('week')['move'])
Quants_ch['w128']=Quants_ch['week'].map(w128_prices_chain.set_index('week')['move'])
Quants_ch['a64']=Quants_ch['week'].map(a64_prices_chain.set_index('week')['move'])
Quants_ch['a128']=Quants_ch['week'].map(a128_prices_chain.set_index('week')['move'])
Quants_ch['s64']=Quants_ch['week'].map(s64_prices_chain.set_index('week')['move'])
Quants_ch=Quants_ch.drop(columns=['week'], axis=1)

opt_price_CH = pd.concat([market4, wholesale_prices_ch, elasticities_ch, Quants_ch], axis=1)

opt_pricesCH_matrix=np.zeros((market4.size,9))

for i in range(market4.shape[0]):
    omegaCH=np.vstack([[opt_price_CH.iloc[i,10:19]],[opt_price_CH.iloc[i,19:28]],[opt_price_CH.iloc[i,28:37]]\
                   ,[opt_price_CH.iloc[i,37:46]],[opt_price_CH.iloc[i,46:55]],[opt_price_CH.iloc[i,55:64]]\
                   ,[opt_price_CH.iloc[i,64:73]],[opt_price_CH.iloc[i,73:82]],[opt_price_CH.iloc[i,82:91]]])
    omegaCH_inv=lin.inv(omegaCH)
    whprCH=np.reshape(np.array(opt_price_CH.iloc[i,1:10]),(9,1)) - \
    omegaCH_inv@np.reshape(np.array(opt_price_CH.iloc[i,91:100]),(9,1))
    opt_pricesCH_matrix[i,:]=whprCH.T
    i+=1

opt_pricesCH_matrix=np.hstack((market4, opt_pricesCH_matrix))
opt_pricesCH=pd.DataFrame(opt_pricesCH_matrix)
opt_pricesCH.columns=['week','t64_c','t128_c', 'c64_c', 'c128_c', 'w64_c', 'w128_c', 'a64_c', 'a128_c', 's64_c']

C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

# Customer Welfare

## Chain vs Store

What did I do?: i) ...

In [87]:
market3.shape

(765, 1)

In [94]:
# Building the store-week dataset.

datapack_cws = np.hstack((zone_week, datapack_GMM[['zone', 'week', 'store', 'store-week','totcount', 'move', 'promo',\
                                            '64 OZ', '128 OZ', 'Tide','cheer','wisk','all_det','surf']]))

datapack_cws=pd.DataFrame(datapack_cws)

datapack_cws.columns = ['zone-week', 'zone', 'week', 'store', 'store-week','totcount', 'move', 'promo',\
                        '64 OZ', '128 OZ', 'Tide','cheer','wisk','all_det','surf']


datapack_cws['income']=datapack_cws['store'].map(zones.set_index('store')['income'])
datapack_cws['AGE60']=datapack_cws['store'].map(zones.set_index('store')['AGE60'])
datapack_cws['ethnic']=datapack_cws['store'].map(zones.set_index('store')['ethnic'])
datapack_cws['hvalmean']=datapack_cws['store'].map(zones.set_index('store')['hvalmean'])
datapack_cws['shopindx']=datapack_cws['store'].map(zones.set_index('store')['shopindx'])
datapack_cws['cubdist']=datapack_cws['store'].map(zones.set_index('store')['cubdist'])
datapack_cws['omnidist']=datapack_cws['store'].map(zones.set_index('store')['omnidist'])
datapack_cws['sstrdist']=datapack_cws['store'].map(zones.set_index('store')['sstrdist'])
datapack_cws['dtdist']=datapack_cws['store'].map(zones.set_index('store')['dtdist'])

datapack_cws['pt64'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['t64_s'])*datapack_cws['64 OZ']*datapack_cws['Tide']

datapack_cws['pt128'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['t128_s'])*datapack_cws['128 OZ']*datapack_cws['Tide']

datapack_cws['pc64'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['c64_s'])*datapack_cws['64 OZ']*datapack_cws['cheer']

datapack_cws['pc128'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['c128_s'])*datapack_cws['128 OZ']*datapack_cws['cheer']

datapack_cws['pw64'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['w64_s'])*datapack_cws['64 OZ']*datapack_cws['wisk']

datapack_cws['pw128'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['w128_s'])*datapack_cws['128 OZ']*datapack_cws['wisk']

datapack_cws['pa64'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['a64_s'])*datapack_cws['64 OZ']*datapack_cws['all_det']

datapack_cws['pa128'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['a128_s'])*datapack_cws['128 OZ']*datapack_cws['all_det']

datapack_cws['ps64'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['s64_s'])*datapack_cws['64 OZ']*datapack_cws['surf']

In [98]:
# Building the zone-week dataset.



datapack_cwz = np.hstack((zone_week, datapack_GMM[['zone', 'week', 'store', 'store-week','totcount', 'move', 'promo',\
                                            '64 OZ', '128 OZ', 'Tide','cheer','wisk','all_det','surf']]))

datapack_cwz=pd.DataFrame(datapack_cwz)

datapack_cwz.columns = ['zone-week', 'zone', 'week', 'store', 'store-week','totcount', 'move', 'promo',\
                        '64 OZ', '128 OZ', 'Tide','cheer','wisk','all_det','surf']



datapack_cwz['income']=datapack_cwz['store'].map(zones.set_index('store')['income'])
datapack_cwz['AGE60']=datapack_cwz['store'].map(zones.set_index('store')['AGE60'])
datapack_cwz['ethnic']=datapack_cwz['store'].map(zones.set_index('store')['ethnic'])
datapack_cwz['hvalmean']=datapack_cwz['store'].map(zones.set_index('store')['hvalmean'])
datapack_cwz['shopindx']=datapack_cwz['store'].map(zones.set_index('store')['shopindx'])
datapack_cwz['cubdist']=datapack_cwz['store'].map(zones.set_index('store')['cubdist'])
datapack_cwz['omnidist']=datapack_cwz['store'].map(zones.set_index('store')['omnidist'])
datapack_cwz['sstrdist']=datapack_cwz['store'].map(zones.set_index('store')['sstrdist'])
datapack_cwz['dtdist']=datapack_cwz['store'].map(zones.set_index('store')['dtdist'])


datapack_cwz['pt64'] = datapack_cwz['zone-week'].map(opt_pricesZN.set_index('zone-week')['t64_z'])*datapack_cwz['64 OZ']*datapack_cwz['Tide']

datapack_cwz['pt128'] = datapack_cwz['zone-week'].map(opt_pricesZN.set_index('zone-week')['t128_z'])*datapack_cwz['128 OZ']*datapack_cwz['Tide']

datapack_cwz['pc64'] = datapack_cwz['zone-week'].map(opt_pricesZN.set_index('zone-week')['c64_z'])*datapack_cwz['64 OZ']*datapack_cwz['cheer']

datapack_cwz['pc128'] = datapack_cwz['zone-week'].map(opt_pricesZN.set_index('zone-week')['c128_z'])*datapack_cwz['128 OZ']*datapack_cwz['cheer']

datapack_cwz['pw64'] = datapack_cwz['zone-week'].map(opt_pricesZN.set_index('zone-week')['w64_z'])*datapack_cwz['64 OZ']*datapack_cwz['wisk']

datapack_cwz['pw128'] = datapack_cwz['zone-week'].map(opt_pricesZN.set_index('zone-week')['w128_z'])*datapack_cwz['128 OZ']*datapack_cwz['wisk']

datapack_cwz['pa64'] = datapack_cwz['zone-week'].map(opt_pricesZN.set_index('zone-week')['a64_z'])*datapack_cwz['64 OZ']*datapack_cwz['all_det']

datapack_cwz['pa128'] = datapack_cwz['zone-week'].map(opt_pricesZN.set_index('zone-week')['a128_z'])*datapack_cwz['128 OZ']*datapack_cwz['all_det']

datapack_cwz['ps64'] = datapack_cwz['zone-week'].map(opt_pricesZN.set_index('zone-week')['s64_z'])*datapack_cwz['64 OZ']*datapack_cwz['surf']

In [103]:
# Building the chain-week dataset.

#d_Tide64=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands,\ 
#P_Tide64_diff, storeweek)

#countf_perweek = countf.groupby(['store','week','store-week'], as_index=False)['custcoun'].sum()

datapack_cwc = np.hstack((zone_week, datapack_GMM[['zone', 'week', 'store', 'store-week','totcount', 'move', 'promo',\
                                            '64 OZ', '128 OZ', 'Tide','cheer','wisk','all_det','surf']]))

datapack_cwc=pd.DataFrame(datapack_cwc)

datapack_cwc.columns = ['zone-week', 'zone', 'week', 'store', 'store-week','totcount', 'move', 'promo',\
                        '64 OZ', '128 OZ', 'Tide','cheer','wisk','all_det','surf']


datapack_cwc['income']=datapack_cwc['store'].map(zones.set_index('store')['income'])#*datapack_cwc['totcount']
datapack_cwc['AGE60']=datapack_cwc['store'].map(zones.set_index('store')['AGE60'])#*datapack_cwc['totcount']
datapack_cwc['ethnic']=datapack_cwc['store'].map(zones.set_index('store')['ethnic'])#*datapack_cwc['totcount']
datapack_cwc['hvalmean']=datapack_cwc['store'].map(zones.set_index('store')['hvalmean'])#*datapack_cwc['totcount']
datapack_cwc['shopindx']=datapack_cwc['store'].map(zones.set_index('store')['shopindx'])#*datapack_cwc['totcount']
datapack_cwc['cubdist']=datapack_cwc['store'].map(zones.set_index('store')['cubdist'])#*datapack_cwc['totcount']
datapack_cwc['omnidist']=datapack_cwc['store'].map(zones.set_index('store')['omnidist'])#*datapack_cwc['totcount']
datapack_cwc['sstrdist']=datapack_cwc['store'].map(zones.set_index('store')['sstrdist'])#*datapack_cwc['totcount']
datapack_cwc['dtdist']=datapack_cwc['store'].map(zones.set_index('store')['dtdist'])#*datapack_cwc['totcount']



datapack_cwc['pt64'] = datapack_cwc['week'].map(opt_pricesCH.set_index('week')['t64_c'])*datapack_cwc['64 OZ']*datapack_cwc['Tide']

datapack_cwc['pt128'] = datapack_cwc['week'].map(opt_pricesCH.set_index('week')['t128_c'])*datapack_cwc['128 OZ']*datapack_cwc['Tide']

datapack_cwc['pc64'] = datapack_cwc['week'].map(opt_pricesCH.set_index('week')['c64_c'])*datapack_cwc['64 OZ']*datapack_cwc['cheer']

datapack_cwc['pc128'] = datapack_cwc['week'].map(opt_pricesCH.set_index('week')['c128_c'])*datapack_cwc['128 OZ']*datapack_cwc['cheer']

datapack_cwc['pw64'] = datapack_cwc['week'].map(opt_pricesCH.set_index('week')['w64_c'])*datapack_cwc['64 OZ']*datapack_cwc['wisk']

datapack_cwc['pw128'] = datapack_cwc['week'].map(opt_pricesCH.set_index('week')['w128_c'])*datapack_cwc['128 OZ']*datapack_cwc['wisk']

datapack_cwc['pa64'] = datapack_cwc['week'].map(opt_pricesCH.set_index('week')['a64_c'])*datapack_cwc['64 OZ']*datapack_cwc['all_det']

datapack_cwc['pa128'] = datapack_cwc['week'].map(opt_pricesCH.set_index('week')['a128_c'])*datapack_cwc['128 OZ']*datapack_cwc['all_det']

datapack_cwc['ps64'] = datapack_cwc['week'].map(opt_pricesCH.set_index('week')['s64_c'])*datapack_cwc['64 OZ']*datapack_cwc['surf']

## Calculating differences in consumer welfare
What did I do?: i) prepare the data to generate the deltas and the mu. ii) calculate the deltas and mu for each configuration. iii) group by store-week. iv) take logs. v) calculate the numerator. vi) divide by $\theta$. vii) take average across simulations

In [105]:
# Preparing the data

#delta_fun (alpha, beta, sigma, gamma, theta, xi, data_promo_size, data_demog, brand_dummies, price, store_week):

#mu_fun(data_promo_size, price, brand_dummies, L_matrix, lambda_x,lambda_price, sim_beta, sim_theta, sim_w):

# draws:

#normal_beta_draws
#normal_theta_draws
#normal_w_draws

# store configuration:

promo_size_s = np.array(datapack_cws[['promo', '64 OZ']])
demog_data_s = np.array(datapack_cws[['income', 'AGE60', 'ethnic', 'hvalmean','shopindx','sstrdist']])
brand_dummies_s = np.array(datapack_cws[['Tide','cheer','wisk','all_det','surf']])
storeweek_s = np.array(datapack_cws['store-week'])

price1_s = datapack_cws[['pt64', 'pt128', 'pc64', 'pc128', 'pw64', 'pw128', 'pa64', 'pa128', 'ps64']]
price2_s = price1_s.sum(axis=1)
price_s = np.array(price2_s, ndmin=2).T

# zone configuration:

promo_size_z = np.array(datapack_cwz[['promo', '64 OZ']])
demog_data_z = np.array(datapack_cwz[['income', 'AGE60', 'ethnic', 'hvalmean','shopindx','sstrdist']])
brand_dummies_z = np.array(datapack_cwz[['Tide','cheer','wisk','all_det','surf']])
storeweek_z = np.array(datapack_cwz['store-week'])

price1_z = datapack_cwz[['pt64', 'pt128', 'pc64', 'pc128', 'pw64', 'pw128', 'pa64', 'pa128', 'ps64']]
price2_z = price1_z.sum(axis=1)
price_z = np.array(price2_z, ndmin=2).T

# chain configuration:

promo_size_c = np.array(datapack_cwc[['promo', '64 OZ']])
demog_data_c = np.array(datapack_cwc[['income', 'AGE60', 'ethnic', 'hvalmean','shopindx','sstrdist']])
brand_dummies_c = np.array(datapack_cwc[['Tide','cheer','wisk','all_det','surf']])
storeweek_c = np.array(datapack_cwc['store-week'])

price1_c = datapack_cwc[['pt64', 'pt128', 'pc64', 'pc128', 'pw64', 'pw128', 'pa64', 'pa128', 'ps64']]
price2_c = price1_c.sum(axis=1)
price_c = np.array(price2_c, ndmin=2).T


In [106]:
# calculating deltas and mu for each configuration:


# draws:

#normal_beta_draws
#normal_theta_draws
#normal_w_draws

# store configuration:

xi_s=np.zeros((datapack_cws.shape[0],1))

delta_s = delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi_s, \
                     promo_size_s, demog_data_s, brand_dummies_s, price_s, storeweek_s)

mu_s = mu_fun(promo_size_s, price_s, brand_dummies_s, L_matrix_def, lambda_p_s_def,\
              lambda_price_def, normal_beta_draws, normal_theta_draws, normal_w_draws)

# zone configuration:

xi_z=np.zeros((datapack_cwz.shape[0],1))

delta_z = delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi_z, \
                     promo_size_z, demog_data_z, brand_dummies_z, price_z, storeweek_z)

mu_z = mu_fun(promo_size_z, price_z, brand_dummies_z, L_matrix_def, lambda_p_s_def,\
              lambda_price_def, normal_beta_draws, normal_theta_draws, normal_w_draws)

# chain configuration:

xi_c=np.zeros((datapack_cwc.shape[0],1))

delta_c = delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi_c, \
                     promo_size_c, demog_data_c, brand_dummies_c, price_c, storeweek_c)

mu_c = mu_fun(promo_size_c, price_c, brand_dummies_c, L_matrix_def, lambda_p_s_def,\
              lambda_price_def, normal_beta_draws, normal_theta_draws, normal_w_draws)

In [126]:
# adding up, taking exponential, grouping by store_week and taking logs

# store configuration

delta_s2=np.array(delta_s[:,1], ndmin=2).T

exp_ut1_s = delta_s2 + mu_s
exp_ut2_s = np.exp(exp_ut1_s)
exp_ut2_s = pd.DataFrame(exp_ut2_s)
exp_ut2_s = 1 + exp_ut2_s
exp_ut2_s['store-week'] = datapack_cws['store-week']
exp_ut3_s = exp_ut2_s.groupby(['store-week']).sum()
exp_ut3_s = np.log(exp_ut3_s)

log_expected_ut_s = exp_ut3_s

# zone configuration

delta_z2=np.array(delta_z[:,1], ndmin=2).T

exp_ut1_z = delta_z2 + mu_z
exp_ut2_z = np.exp(exp_ut1_z)
exp_ut2_z = pd.DataFrame(exp_ut2_z)
exp_ut2_z = 1 + exp_ut2_z
exp_ut2_z['store-week'] = datapack_cwz['store-week']
exp_ut3_z = exp_ut2_z.groupby(['store-week']).sum()
exp_ut3_z = np.log(exp_ut3_z)

log_expected_ut_z = exp_ut3_z

# chain configuration

delta_c2=np.array(delta_c[:,1], ndmin=2).T

exp_ut1_c = delta_c2 + mu_c
exp_ut2_c = np.exp(exp_ut1_c)
exp_ut2_c = pd.DataFrame(exp_ut2_c)
exp_ut2_c = 1 + exp_ut2_c
exp_ut2_c['store-week'] = datapack_cwc['store-week']
exp_ut3_c = exp_ut2_c.groupby(['store-week']).sum()
exp_ut3_c = np.log(exp_ut3_c)

log_expected_ut_c = exp_ut3_c

#theta vector (1 row x 30 columns)

gamma = np.reshape(gamma_def, (gamma_def.size,1))
normal_theta_draws = np.reshape(normal_theta_draws,(1, normal_theta_draws.size))

theta_vec = -theta_def + demog_data_s@gamma + lambda_price_def*normal_theta_draws
#review dimensions of theta vec

In [129]:
log_expected_ut_c.shape

(3905, 30)

In [150]:
# calculating differences in consumer welfare per store week, and integrating across simulations


nsim=normal_theta_draws.shape[1]
Mt = np.array([datapack_GMM['totcount']], ndmin=2).T

i=0
Comp_Variation_sc = np.zeros((Mt.size, 1))#[0]*log_expected_ut_c.shape[0]

for i in range(log_expected_ut_c.shape[0]):
    Comp_Variation_sc[i] = ((log_expected_ut_s.iloc[i,:]-log_expected_ut_c.iloc[i,:])/theta_vec[i,:]).sum()*(1/nsim)
    i+=1

i=0
Comp_Variation_zc = np.zeros((Mt.size, 1))#[0]*log_expected_ut_c.shape[0]

for i in range(log_expected_ut_c.shape[0]):
    Comp_Variation_zc[i] = ((log_expected_ut_z.iloc[i,:]-log_expected_ut_c.iloc[i,:])/theta_vec[i,:]).sum()*(1/nsim)
    i+=1

#DROPPING NAN AND INF VALUES

CV_CS = Comp_Variation_sc*Mt
CV_CS = pd.DataFrame(CV_CS)
CV_CS = CV_CS.replace([np.inf, -np.inf], np.nan)
CV_CS = CV_CS.dropna()
CV_CS = np.array(CV_CS)
Comp_Var_CS = CV_CS.sum()

CV_CZ = Comp_Variation_zc*Mt
CV_CZ = pd.DataFrame(CV_CZ)
CV_CZ = CV_CZ.replace([np.inf, -np.inf], np.nan)
CV_CZ = CV_CZ.dropna()
CV_CZ = np.array(CV_CZ)
Comp_Var_CZ = CV_CZ.sum()


In [151]:
print("Compensated Variation from Chain to Store:",Comp_Var_CS)
print("Compensated Variation from Chain toZone:",Comp_Var_CZ)

Compensated Variation from Chain to Store: 1051256.99877796
Compensated Variation from Chain toZone: 476.62973688096224
